<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Loading-data" data-toc-modified-id="Loading-data-1">Loading data</a></span></li><li><span><a href="#DEFUNCT:-Loading-data,-duplicates" data-toc-modified-id="DEFUNCT:-Loading-data,-duplicates-2">DEFUNCT: Loading data, duplicates</a></span><ul class="toc-item"><li><span><a href="#Check,-remove-duplicates" data-toc-modified-id="Check,-remove-duplicates-2.1">Check, remove duplicates</a></span></li></ul></li><li><span><a href="#Simple-column-level-transforms" data-toc-modified-id="Simple-column-level-transforms-3">Simple column-level transforms</a></span></li><li><span><a href="#Create-Allelic-requirement-mapping" data-toc-modified-id="Create-Allelic-requirement-mapping-4">Create Allelic requirement mapping</a></span></li><li><span><a href="#Create-molecular_mechanism-mapping" data-toc-modified-id="Create-molecular_mechanism-mapping-5">Create molecular_mechanism mapping</a></span></li><li><span><a href="#Confidence-values" data-toc-modified-id="Confidence-values-6">Confidence values</a></span><ul class="toc-item"><li><span><a href="#Removing-rows-+-stats" data-toc-modified-id="Removing-rows-+-stats-6.1">Removing rows + stats</a></span></li></ul></li><li><span><a href="#Pre-NodeNorming" data-toc-modified-id="Pre-NodeNorming-7">Pre-NodeNorming</a></span><ul class="toc-item"><li><span><a href="#Exploring:-Genes" data-toc-modified-id="Exploring:-Genes-7.1">Exploring: Genes</a></span><ul class="toc-item"><li><span><a href="#HGNC" data-toc-modified-id="HGNC-7.1.1">HGNC</a></span></li><li><span><a href="#OMIM" data-toc-modified-id="OMIM-7.1.2">OMIM</a></span></li><li><span><a href="#Comparing-HGNC-vs-OMIM" data-toc-modified-id="Comparing-HGNC-vs-OMIM-7.1.3">Comparing HGNC vs OMIM</a></span></li><li><span><a href="#Conclusions" data-toc-modified-id="Conclusions-7.1.4">Conclusions</a></span></li></ul></li><li><span><a href="#Exploring:-Diseases" data-toc-modified-id="Exploring:-Diseases-7.2">Exploring: Diseases</a></span><ul class="toc-item"><li><span><a href="#OMIM/orphanet" data-toc-modified-id="OMIM/orphanet-7.2.1">OMIM/orphanet</a></span></li><li><span><a href="#MONDO" data-toc-modified-id="MONDO-7.2.2">MONDO</a></span></li><li><span><a href="#Comparing-OMIM/orphanet-vs-MONDO" data-toc-modified-id="Comparing-OMIM/orphanet-vs-MONDO-7.2.3">Comparing OMIM/orphanet vs MONDO</a></span></li><li><span><a href="#Checking-MONDO-data" data-toc-modified-id="Checking-MONDO-data-7.2.4">Checking MONDO data</a></span></li><li><span><a href="#Conclusions" data-toc-modified-id="Conclusions-7.2.5">Conclusions</a></span></li></ul></li></ul></li><li><span><a href="#Stats-on-rows-removed-during-NodeNorming" data-toc-modified-id="Stats-on-rows-removed-during-NodeNorming-8">Stats on rows removed during NodeNorming</a></span></li><li><span><a href="#Adding-NodeNorm-data,-removing-rows" data-toc-modified-id="Adding-NodeNorm-data,-removing-rows-9">Adding NodeNorm data, removing rows</a></span></li><li><span><a href="#Generating-documents" data-toc-modified-id="Generating-documents-10">Generating documents</a></span><ul class="toc-item"><li><span><a href="#Rows-not-included" data-toc-modified-id="Rows-not-included-10.1">Rows not included</a></span></li><li><span><a href="#Columns-not-included" data-toc-modified-id="Columns-not-included-10.2">Columns not included</a></span></li><li><span><a href="#BioThings-type-parser" data-toc-modified-id="BioThings-type-parser-10.3">BioThings-type parser</a></span></li><li><span><a href="#File:-List-of-TRAPI-edges" data-toc-modified-id="File:-List-of-TRAPI-edges-10.4">File: List of TRAPI edges</a></span></li><li><span><a href="#File:-KGX-edges" data-toc-modified-id="File:-KGX-edges-10.5">File: KGX edges</a></span></li><li><span><a href="#File:-KGX-nodes" data-toc-modified-id="File:-KGX-nodes-10.6">File: KGX nodes</a></span></li></ul></li><li><span><a href="#DEFUNCT:-Checking-documents" data-toc-modified-id="DEFUNCT:-Checking-documents-11">DEFUNCT: Checking documents</a></span></li><li><span><a href="#BioThings-Parser-notes" data-toc-modified-id="BioThings-Parser-notes-12">BioThings Parser notes</a></span></li></ul></div>

# Notebook for EBI gene2pheno parser development

In [1]:
## not for parser. for notebook only 

## CX: allows multiple lines of code to print from one code block
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Loading data

In [2]:
## put into parser: DONE
import pathlib
import pandas as pd
import requests
from datetime import datetime

## don't put in parser. Just for this notebook
import glob
from pprint import pprint

## unsure on putting into parser: more for notebook viewing/debugging...
pd.options.display.max_columns = None
# pd.set_option('display.max_colwidth', None)
# pd.reset_option('display.max_colwidth')

**UPDATE 2025-07-31**

Now using the "on-the-fly", "all data" download thru the API. I think this is the easiest to set up the common ingest pipeline to use. It has all data **without duplicates** (analyzed in data-playground notebook).

Putting old notes/code chunks that used static release panel files in a separate section.

In [3]:
## get current time of download for common ingest pipeline
## from https://stackoverflow.com/a/69080715 
datetime.now(datetime.now().astimezone().tzinfo).isoformat()
datetime.now(datetime.now().astimezone().tzinfo).strftime("%Y-%m-%dT%H:%M:%S.%f%:z")

'2025-09-04T12:37:11.814360-07:00'

'2025-09-04T12:37:11.818547-07:00'

In [4]:
## put into parser (format): 

all_data_url = "https://www.ebi.ac.uk/gene2phenotype/api/panel/all/download/"

## ingesting all columns as str for now
df = pd.read_csv(all_data_url, dtype=str)

## make column names snake-case - usable with itertuples later
df.columns = df.columns.str.replace(" ", "_")

In [5]:
df.columns

Index(['g2p_id', 'gene_symbol', 'gene_mim', 'hgnc_id', 'previous_gene_symbols',
       'disease_name', 'disease_mim', 'disease_MONDO', 'allelic_requirement',
       'cross_cutting_modifier', 'confidence', 'variant_consequence',
       'variant_types', 'molecular_mechanism', 'molecular_mechanism_support',
       'molecular_mechanism_categorisation', 'molecular_mechanism_evidence',
       'phenotypes', 'publications', 'panel', 'comments',
       'date_of_last_review', 'review'],
      dtype='object')

In [6]:
## looking at datatype of this column
df["date_of_last_review"].info(memory_usage="deep")

<class 'pandas.core.series.Series'>
RangeIndex: 3720 entries, 0 to 3719
Series name: date_of_last_review
Non-Null Count  Dtype 
--------------  ----- 
3720 non-null   object
dtypes: object(1)
memory usage: 269.0 KB


In [7]:
## change this column to datetime, saves memory
df["date_of_last_review"] = pd.to_datetime(df["date_of_last_review"])
df["date_of_last_review"].info(memory_usage="deep")

<class 'pandas.core.series.Series'>
RangeIndex: 3720 entries, 0 to 3719
Series name: date_of_last_review
Non-Null Count  Dtype              
--------------  -----              
3720 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1)
memory usage: 29.2 KB


In [8]:
## check for duplicates - empty df == none

df[df.duplicated(keep=False)]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_support,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,review


In [9]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3720 entries, 0 to 3719
Data columns (total 23 columns):
 #   Column                              Non-Null Count  Dtype              
---  ------                              --------------  -----              
 0   g2p_id                              3720 non-null   object             
 1   gene_symbol                         3720 non-null   object             
 2   gene_mim                            3717 non-null   object             
 3   hgnc_id                             3720 non-null   object             
 4   previous_gene_symbols               3435 non-null   object             
 5   disease_name                        3720 non-null   object             
 6   disease_mim                         2954 non-null   object             
 7   disease_MONDO                       2224 non-null   object             
 8   allelic_requirement                 3720 non-null   object             
 9   cross_cutting_modifier              468 n

In [10]:
df

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_support,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,review
0,G2P00001,HMX1,142992,5017,H6; NKX5-3,HMX1-related oculoauricular syndrome,612109,MONDO:0012802,biallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000007; HP:0000482; HP:0000505; HP:0000518...,18423520; 25574057; 29140751,DD; Eye,NaN,2019-09-26 16:23:46+00:00,NaN
1,G2P00002,SLX4,613278,23845,BTBD12; FANCP; KIAA1784; KIAA1987,SLX4-related Fanconi anemia,613951,MONDO:0013499,biallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000007; HP:0000028; HP:0000085; HP:0000125...,21240275; 21240277,DD,NaN,2025-01-28 23:09:54+00:00,NaN
2,G2P00003,ARG1,608313,663,NaN,ARG1-related argininemia,207800,MONDO:0008814,biallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000007; HP:0000737; HP:0000752; HP:0001249...,10502833; 1463019; 1598908; 2365823; 7649538,DD,NaN,2015-07-22 16:14:07+00:00,NaN
3,G2P00004,ATR,601215,882,FRP1; MEC1; SCKL; SCKL1,ATR-related Seckel syndrome,210600,MONDO:0008869,biallelic_autosomal,NaN,strong,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000007; HP:0000028; HP:0000047; HP:0000057...,NaN,DD; Skeletal,NaN,2025-01-27 14:24:27+00:00,NaN
4,G2P00005,FANCB,300515,3583,FAAP95; FAB; FLJ34064,FANCB-related Fanconi anemia,300514,MONDO:0010351,monoallelic_X_hemizygous,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000083; HP:0000100; HP:0000119; HP:0000707...,15502827; 16679491; 21910217; 22052692; 236135...,DD; Skin,NaN,2024-08-20 14:13:58+00:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3715,G2P01393,CABP2,607314,1385,DFNB93,CABP2-related deafness,614899,NaN,biallelic_autosomal,NaN,strong,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000007,22981119,Ear,NaN,2025-04-28 09:36:23+00:00,NaN
3716,G2P01854,PAX3,606597,8617,HUP2; PAX-3; WS1,PAX3-related craniofacial-deafness-hand syndrome,122880,NaN,monoallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000006; HP:0000272; HP:0000316; HP:0000327...,6859126,Ear,NaN,2025-04-28 10:19:36+00:00,NaN
3717,G2P01857,SIX1,601205,10887,DFNA23,SIX1-related deafness,605192,NaN,monoallelic_autosomal,NaN,definitive,altered gene product structure,inframe_deletion; inframe_insertion; missense_...,undetermined,inferred,NaN,NaN,HP:0000006; HP:0000405; HP:0000407,10777717,Ear,NaN,2025-04-28 10:21:02+00:00,NaN
3718,G2P01852,SOX10,602229,11190,DOM; SOX-10; WS2E; WS4,SOX10-related Kallmann syndrome with deafness,NaN,NaN,monoallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,NaN,23643381,Ear,NaN,2025-04-28 10:21:28+00:00,NaN


## DEFUNCT: Loading data, duplicates

<div class="alert alert-block alert-danger">
    
Adjust the code block below for path/pattern for data files. 
    
This notebook was originally written using data files from the 2025-02-28 static release on the [FTP site](https://ftp.ebi.ac.uk/pub/databases/gene2phenotype/G2P_data_downloads/).
The latest data can be downloaded from the [website](https://www.ebi.ac.uk/gene2phenotype/download) or [API](https://www.ebi.ac.uk/gene2phenotype/api/)

In [17]:
# ## put into parser (format): DONE

# base_file_path = pathlib.Path.home().joinpath("Desktop", "EBIgene2pheno_files", 
#                                               "From_FTP", "2025-06-27")

# ## uses pathlib's Path.glob, which produces a generator. 
# ## cast into list so parser code can check if paths were actually matched or not
# all_file_paths = list(base_file_path.glob("*.csv.gz"))
# all_file_paths

In [18]:
# ## an example: pathlib's Path.glob produces a generator
# ## vs glob.glob produces an array (from cwd only?)
# base_file_path.glob("*2025-02-28.csv.gz")
# glob.glob("*2025-02-28.csv.gz")

__OLD NOTES__


Approach is to load all files into 1 pandas dataframe. Then I can...

1. check the duplicates situation: records found in multiple panel files. I can check whether the same record looks different between files or not (by checking duplicates using all columns vs key columns). -> Raise errors if yes
2. remove duplicates before generating documents
3. Do some tasks column-wise over all the data, rather than while iterating over rows

Notes:
* There are a few existing BioThings parsers that also use `pandas` to load the entire raw data file at once: https://github.com/search?q=repo%3Abiothings%2Fpending.api%20pandas&type=code
* But there are other parsers that use `csv` to load the file **one row at a time** (generator): https://github.com/search?q=repo%3Abiothings%2Fpending.api+csv+reader&type=code

---

If I did the generator approach (load files 1 by 1, 1 row at a time), I'd have to modify how I do things:
1. Don't do the duplicates check. But try to mitigate potential "duplicate" issues: 
   - Sort all delimited strings
   - Use a hash of all column values (when they're all strings) for `_id`. Want rows with all the same values to produce the same hash
2. Either leave to BioThings toolset to remove duplicates, or could save a running set of `_id` hashes to check if row was already encountered -> not create duplicate docs
3. Do the tasks on single rows/chunks (pandas [read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html#pandas.read_csv) has an iterator for rows/chunks! see iterator/chunksize parameters)

In [19]:
# ## put into parser (format): DONE

# ## using generator expression (think list/dict comprehension) within pd.concat to load files 1 at a time
# ## ingesting all columns as str for now
# df = pd.concat((pd.read_csv(f, dtype=str) for f in all_file_paths), ignore_index=True)

# ## make column names snake-case - usable with itertuples later
# df.columns = df.columns.str.replace(" ", "_")

In [20]:
# df["date_of_last_review"].info(memory_usage="deep")

In [21]:
# ## change this column to datetime, saves memory
# df["date_of_last_review"] = pd.to_datetime(df["date_of_last_review"])
# df["date_of_last_review"].info(memory_usage="deep")

In [22]:
# # I couldn't figure out how to import + ingest column as datetime in 1 step 
# # this is what I tried that didn't work

# # worked with pandas 2.0.3, but didn't work with pandas 2.2.3: ingested as str
# df = pd.concat((pd.read_csv(f, dtype=str, parse_dates=["date of last review"]) 
#                 for f in all_file_paths), ignore_index=True)

# # doesn't work
# df = pd.concat((pd.read_csv(f, dtype=str, parse_dates=["date of last review"], 
#                            date_format="%Y-%m-%d %H:%M:%S%:z") 
#                 for f in all_file_paths), ignore_index=True)
# # throws an error
# df = pd.concat((pd.read_csv(f, dtype=str, parse_dates=[["date of last review"]], 
#                            date_format="%Y-%m-%d %H:%M:%S%:z") 
#                 for f in all_file_paths), ignore_index=True)
# # throws an error
# df = pd.concat((pd.read_csv(f, dtype={"date of last review": pd.datetime64[ns, tz]})
#                 for f in all_file_paths), ignore_index=True)

In [23]:
# df.shape
# df.head()

In [24]:
# df.info(memory_usage="deep")

### Check, remove duplicates

There are duplicate rows in this dataframe because the record (gene + disease + more) is in several panels (disease falls into multiple categories). This was explored in the data-playground notebook. 

We want to drop those duplicates. 
However, I was concerned that the delimited-string values could differ (only in list order) for the same record in diff files. 
So that's what this check is for. 

In [25]:
# ## put into parser (format): DONE

# n_duplicates_column_combo = df[df.duplicated(subset=["g2p_id"], keep=False)].shape

# n_duplicates_all_columns = df[df.duplicated(keep=False)].shape

# ## for testing
# # n_duplicates_all_columns = (1, 1)


# if n_duplicates_column_combo != n_duplicates_all_columns: 
#     raise AssertionError("The data format has changed, and the assumptions about duplicates/key columns may " \
#                           "no longer hold. Re-explore the data and adjust the parser.")

In [26]:
# ## put into parser (format): DONE

# ## drop duplicates
# df.drop_duplicates(inplace=True, ignore_index=True)

In [27]:
# df.info(memory_usage="deep")

## Simple column-level transforms

Based on data-playground "Notes on parsing data to create documents" section

disease_mim column includes both omim and orphanet IDs right now

In [11]:
df[df["disease_mim"].str.contains("Orphanet:", na=False)]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_support,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,review
1964,G2P02564,TONSL,604546,7801,IKBR; NFKBIL2,TONSL-related sponastrime dysplasia,Orphanet:93357,NaN,biallelic_autosomal,NaN,strong,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0002650; HP:0004322; HP:0005281; HP:0100255,30773277; 30773278,DD,NaN,2018-11-07 09:53:40+00:00,NaN


In [29]:
# ## double-checking how to add prefixes to OMIM vs orphanet IDs

# df_diseasemim = df.copy()

# ## done to preserve NA
# df_diseasemim["disease_mim"] = [i if pd.isna(i) \
#                                 else "OMIM:" + i if i.isnumeric() \
#                                 else i \
#                                 for i in df_diseasemim["disease_mim"]]

# df_diseasemim["disease_mim"] = df_diseasemim["disease_mim"].str.replace("Orphanet", "orphanet")

# df_diseasemim[df_diseasemim["disease_mim"].str.contains("OMIM:", na=False)].shape

# df_diseasemim[df_diseasemim["disease_mim"].str.contains("orphanet:", na=False)].shape

# ## add up row count. If == num non-null in info above, you're good 
# ## right now 2944 == 2944, so good

In [12]:
## put into parser (format): 

## COLUMN-LEVEL TRANSFORMS

## adding Translator/biolink prefixes to IDs
df["gene_mim"] = "OMIM:" + df["gene_mim"]
df["hgnc_id"] = "HGNC:" + df["hgnc_id"]
# df["disease_mim"] = df["disease_mim"].str.replace("Orphanet", "orphanet")
## done to preserve NA
df["disease_mim"] = [i if pd.isna(i)
                     else "OMIM:" + i if i.isnumeric()
                     else i.lower()
                     for i in df["disease_mim"]]

## strip whitespace
df["disease_name"] = df["disease_name"].str.strip()
df["comments"] = df["comments"].str.strip()

## create new columns
## UI really wants resource website urls like this. May need to adjust over time as website changes
df["g2p_record_url"] = "https://www.ebi.ac.uk/gene2phenotype/lgd/" +  df["g2p_id"]

## replace panel keywords with full names shown on G2P website for single record
df["panel"] = df["panel"].str.replace("Cancer", "Cancer disorders")
df["panel"] = df["panel"].str.replace("Cardiac", "Cardiac disorders")
df["panel"] = df["panel"].str.replace("DD", "Developmental disorders")
df["panel"] = df["panel"].str.replace("Ear", "Ear disorders")
df["panel"] = df["panel"].str.replace("Eye", "Eye disorders")
df["panel"] = df["panel"].str.replace("Skeletal", "Skeletal disorders")
df["panel"] = df["panel"].str.replace("Skin", "Skin disorders")

In [13]:
## checking on column-level transforms

df.head()
# df["g2p record url"].unique()[0:100]

df[df["disease_mim"].str.contains("orphanet", na=False)]
# df[df["panel"].str.contains("Ear", na=False)]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_support,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,review,g2p_record_url
0,G2P00001,HMX1,OMIM:142992,HGNC:5017,H6; NKX5-3,HMX1-related oculoauricular syndrome,OMIM:612109,MONDO:0012802,biallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000007; HP:0000482; HP:0000505; HP:0000518...,18423520; 25574057; 29140751,Developmental disorders; Eye disorders,NaN,2019-09-26 16:23:46+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00001
1,G2P00002,SLX4,OMIM:613278,HGNC:23845,BTBD12; FANCP; KIAA1784; KIAA1987,SLX4-related Fanconi anemia,OMIM:613951,MONDO:0013499,biallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000007; HP:0000028; HP:0000085; HP:0000125...,21240275; 21240277,Developmental disorders,NaN,2025-01-28 23:09:54+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00002
2,G2P00003,ARG1,OMIM:608313,HGNC:663,NaN,ARG1-related argininemia,OMIM:207800,MONDO:0008814,biallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000007; HP:0000737; HP:0000752; HP:0001249...,10502833; 1463019; 1598908; 2365823; 7649538,Developmental disorders,NaN,2015-07-22 16:14:07+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00003
3,G2P00004,ATR,OMIM:601215,HGNC:882,FRP1; MEC1; SCKL; SCKL1,ATR-related Seckel syndrome,OMIM:210600,MONDO:0008869,biallelic_autosomal,NaN,strong,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000007; HP:0000028; HP:0000047; HP:0000057...,NaN,Developmental disorders; Skeletal disorders,NaN,2025-01-27 14:24:27+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00004
4,G2P00005,FANCB,OMIM:300515,HGNC:3583,FAAP95; FAB; FLJ34064,FANCB-related Fanconi anemia,OMIM:300514,MONDO:0010351,monoallelic_X_hemizygous,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000083; HP:0000100; HP:0000119; HP:0000707...,15502827; 16679491; 21910217; 22052692; 236135...,Developmental disorders; Skin disorders,NaN,2024-08-20 14:13:58+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00005


,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_support,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,review,g2p_record_url
1964,G2P02564,TONSL,OMIM:604546,HGNC:7801,IKBR; NFKBIL2,TONSL-related sponastrime dysplasia,orphanet:93357,NaN,biallelic_autosomal,NaN,strong,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0002650; HP:0004322; HP:0005281; HP:0100255,30773277; 30773278,Developmental disorders,NaN,2018-11-07 09:53:40+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02564


## Create Allelic requirement mapping

**2025-08-06:**

The [allelic requirement edge property/attribute](https://github.com/biolink/biolink-model/pull/1576) has been added to biolink-model, so we can add this column's data to the Translator output. The values are supposed to be HP terms, so we need to map [EBI Gene2Phenotype's allelic_requirement values](https://www.ebi.ac.uk/gene2phenotype/about/terminology#allelic-requirement-section) to their corresponding HP terms.

I'm going to map all possible allelic_requirement values in [EBI Gene2Phenotype's terminology page](https://www.ebi.ac.uk/gene2phenotype/about/terminology#allelic-requirement-section), even if they don't show up in the data right now. **I'll be creating a mapping dictionary that's used when creating the associations later.**

I'm using **OLS API** for **most of the mappings** - looking up in the current HP ontology what term has the allelic_requirement value as a synonym. Note that this OLS endpoint can't query on or show the type of synonym (ref: [github issue](https://github.com/EBISPOT/ols4/issues/917)). The API docs are [here](https://www.ebi.ac.uk/ols4/help) (they're long and confusing). 

**Then I'm doing some mappings manually** because the HP synonyms are incorrect or missing (made a [github issue for HP](https://github.com/obophenotype/human-phenotype-ontology/issues/11243)). 

*Note: I'm not using the new [HP API](https://ontology.jax.org/api/hp/docs) because I found it unreliable: have to replace("_", " ") allelic_requirement values to search, then search results include many incorrect other IDs.*

In [14]:
## actual values in data
df["allelic_requirement"].value_counts()

allelic_requirement
biallelic_autosomal           1991
monoallelic_autosomal         1484
monoallelic_X_hemizygous       176
monoallelic_X_heterozygous      59
mitochondrial                    9
monoallelic_Y_hemizygous         1
Name: count, dtype: int64

In [15]:
## requires requests library

def retrieve_allelicreq_mappings(allelic_req_val):
    ols_request = f"https://www.ebi.ac.uk/ols4/api/search?q={allelic_req_val}&ontology=hp&queryFields=synonym&exact=true"
    try:
        response = requests.get(ols_request, timeout=5)

        if response.status_code == 200:
            temp = response.json()
            ## only need the HP ID
            return temp["response"]["docs"][0]["obo_id"]
        else:
            print(f"Error retrieving {allelic_req_val}: {response.status_code}")
    except requests.RequestException as e:
        print(f"Problem retrieving {allelic_req_val}. Request exemption: {e}")

In [18]:
## only the values for dynamic mappings - HP has correct synonyms

allelicreq_values_auto = [
    "biallelic_autosomal",
    "monoallelic_autosomal",
    "biallelic_PAR",
    "monoallelic_PAR",
    "mitochondrial",
    "monoallelic_Y_hemizygous",
    "monoallelic_X",
    "monoallelic_X_hemizygous",
    "monoallelic_X_heterozygous",
]

In [19]:
## using dictionary comprehension
allelicreq_mappings = {i: retrieve_allelicreq_mappings(i) for i in allelicreq_values_auto}

allelicreq_mappings

{'biallelic_autosomal': 'HP:0000007',
 'monoallelic_autosomal': 'HP:0000006',
 'biallelic_PAR': 'HP:0034341',
 'monoallelic_PAR': 'HP:0034340',
 'mitochondrial': 'HP:0001427',
 'monoallelic_Y_hemizygous': 'HP:0001450',
 'monoallelic_X': 'HP:0001417',
 'monoallelic_X_hemizygous': 'HP:0001419',
 'monoallelic_X_heterozygous': 'HP:0001423'}

<div class="alert alert-block alert-danger">

Check for accuracy against my manually-reviewed mappings (from data-playground notebook)
    
| G2P | HPO name | HPO ID | Exact synonym? | 
| :- | :- | :- | :- |
| biallelic_autosomal | Autosomal recessive inheritance | HP:0000007 | Yes |
| monoallelic_autosomal | Autosomal dominant inheritance | HP:0000006 | Yes |
| biallelic_PAR | Pseudoautosomal recessive inheritance | HP:0034341 | Yes |
| monoallelic_PAR | Pseudoautosomal dominant inheritance | HP:0034340 | Yes |
| mitochondrial | Mitochondrial inheritance | HP:0001427 | Yes |
| monoallelic_Y_hemizygous | Y-linked inheritance | HP:0001450 | Yes |
| **monoallelic_X** | X-linked inheritance | HP:0001417 | Yes |
| **monoallelic_X_hemizygous** | X-linked recessive inheritance | HP:0001419 | Yes |
| **monoallelic_X_heterozygous** | X-linked dominant inheritance | HP:0001423 | Yes |

## Create molecular_mechanism mapping

**FOR NOW**, using `subject_form_or_variant_qualifier` (for Gene). 

Values are `genetic_variant_form` or a descendant, based on the molecular_mechanism values. The molecular_mechanism [terms](https://www.ebi.ac.uk/gene2phenotype/about/terminology#molecular-mechanism-section) describe the genotype (of gene variants) linked to the disease. 

I manually create these mappings using the definitions from the EBI Gene2Pheno terminology page - it's straightforward.

Related biolink-model change: https://github.com/biolink/biolink-model/pull/1575

In [20]:
## code chunk to review data

df["molecular_mechanism"].value_counts()

molecular_mechanism
loss of function                     2225
undetermined                         1172
gain of function                      188
dominant negative                     130
undetermined non-loss-of-function       5
Name: count, dtype: int64

In [21]:
molecularmech_mappings = {
    "loss of function": "loss_of_function_variant_form",
    "undetermined": "genetic_variant_form",
    "gain of function": "gain_of_function_variant_form",
    "dominant negative": "dominant_negative_variant_form",
    "undetermined non-loss-of-function": "non_loss_of_function_variant_form"
}

## Confidence values

**2025-07-22:**

Every row/record has 1 confidence value, representing how confident the curators are that "this gene has a causal role in this disease". The definitions of the possible values are provided [here](https://www.ebi.ac.uk/gene2phenotype/about/terminology#g2p-confidence-section). 


**CURRENT DEFINITIONS** (including in case they change later)

> **definitive**: The role of this gene in this particular disease has been repeatedly demonstrated in both the research and clinical diagnostic settings, and has been upheld over time (at least 2 independent publication over 3 years' time). No convincing evidence has emerged that contradicts the role of the gene in the specified disease. (previously labelled as confirmed) The strength of evidence within publications as well as their number and publication dates is taken into account. In practice, this usually means at least 4 publications over 5 years. Typically this will also include convincing bioinformatic or functional evidence of causation, making it very unlikely that this gene-disease association would ever be refuted.
>
>**strong**: The role of this gene as a monogenic cause of disease has been repeatedly and independently demonstrated providing very strong convincing evidence in humans and no conflicting evidence for this gene's role in this disease. (previously labelled as probable).
>
>**moderate**: There is moderate evidence in humans to support a casual role for this gene in this disease with no contradictory evidence. The body of evidence is not large (e.g possibly only one key paper) but appears convincing enough that the gene-disease pair is likely to be validated with additional evidence in the near future.
>
>**limited**: Little human evidence exists to support a casual role for this gene in this disease, but not all evidence has been refuted. For example, there may be a collection of rare missense variants in humans but without convincing functional impact, segregration data that could either arise by chance (e.g across one or two meioses) or does not implicate a single gene, or functional data without direct recapitulation of the phenotype. Overall, the body of evidence does not meet contemporary criteria for claiming a valid association with disease. The majority are probably false associations. (previously labelled as possible).
>
>**disputed**: Although evidence has been reported, other evidence of equal weight disputes the claim.
>
>**refuted**: There has been an assertion of a gene-disease association in the literature, but new valid evidence has arisen that refutes the entire original body of evidence.

<div class="alert alert-block alert-success">

**2025-07-22:**

After input from Sierra and Matt:
1. **FOR NOW**, rows with **"refuted"** or **"disputed"** values **should not be used** to create edges for Translator. These mean there's strong evidence that there ISN'T an association (negation). **This decision can be revisited** once Translator can model/handle negation better. 
2. rows with **"limited"** value **should not be used** to create edges for Translator. Sierra and Matt pointed out the last sentence of the definition: "The majority are probably false associations. (previously labeled as possible)." The reasoning is that these may not be "real" associations. 
3. keep rows with **"moderate", "strong", "definitive"** values, because there's moderate-definitive evidence that a gene DOES HAVE a causal role in this disease. 

<div class="alert alert-block alert-danger">

Data-modeling notes: options for gene-disease associations are confusing 
* "causes / contributes to" makes more sense when qualifiers on the gene/protein are used. 
* what's the diff between "associated with" and "genetically associated with"? 
* "gene associated with condition" is child of "genetically associated with", but seems to be more general - basically a "related to". Also would look weird in UI, right? 

In [22]:
df["confidence"].value_counts()

confidence
definitive    2053
strong         864
limited        529
moderate       273
refuted          1
Name: count, dtype: int64

### Removing rows + stats

In [23]:
## put into parser (format): 

## calculate stats before removing

n_rows_original = df.shape[0]
n_rows_refuted = df[df["confidence"] == "refuted"].shape[0]
n_rows_disputed = df[df["confidence"] == "disputed"].shape[0]
n_rows_limited = df[df["confidence"] == "limited"].shape[0]

In [24]:
## put into parser (format): 

## remove rows, calculate stats after

df = df[~ df["confidence"].isin(["refuted", "disputed", "limited"])].reset_index(drop=True)
n_rows_after_confidence = df.shape[0]

In [25]:
## put into parser (format): 

## Print stats

print(f"{n_rows_original} unique rows/records in original dataset\n")

print(f"Removing rows based on confidence:")
print(f"{n_rows_refuted}: 'refuted'")
print(f"{n_rows_disputed}: 'disputed'")
print(f"{n_rows_limited}: 'limited'\n")


print(f"{n_rows_after_confidence} rows afterwards")

3720 unique rows/records in original dataset

Removing rows based on confidence:
1: 'refuted'
0: 'disputed'
529: 'limited'

3190 rows afterwards


## Pre-NodeNorming

Querying NodeNorm: send unique values (no duplicates!) from entire column in large batches -> generate mapping dict to use. 
<br>
__Not querying 1-by-1 or 1 row at a time: much slower__ and would involve sending duplicate IDs (unless saved dict is kept outside loop and checked) 

Not going to use NameResolver: not optimistic this would work anyways. My manual process of getting "better" disease IDs is to use the gene IDs, find the diseases they're linked to in OMIM and Monarch, and seeing if those match the data's disease name / phenotypes / publications. This is more complicated than just using NameResolver.

<div class="alert alert-block alert-danger">

Set the NodeNorm URL you want to use. 

In [26]:
## put into parser (format): DONE


## from BioThings annotator code: for interoperability between diff Python versions
try:
    from itertools import batched  # new in Python 3.12
except ImportError:
    from itertools import islice

    def batched(iterable, n):
        # batched('ABCDEFG', 3) → ABC DEF G
        if n < 1:
            raise ValueError("n must be at least one")
        iterator = iter(iterable)
        while batch := tuple(islice(iterator, n)):
            yield batch

nodenorm_url = "https://nodenorm.ci.transltr.io/get_normalized_nodes"

### Exploring: Genes

**2025-07-31 data:** Every row has at least 1 gene ID (HGNC column has no missing values). So no rows will be removed because there's no gene IDs to use for the pre-NodeNorming. 

In [27]:
df[["gene_symbol", "hgnc_id", "gene_mim"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3190 entries, 0 to 3189
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   gene_symbol  3190 non-null   object
 1   hgnc_id      3190 non-null   object
 2   gene_mim     3189 non-null   object
dtypes: object(3)
memory usage: 74.9+ KB


In [28]:
df[df["gene_mim"].isna()]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_support,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,review,g2p_record_url
2406,G2P03745,RNU2-2P,NaN,HGNC:10152,NaN,RNU2-2P-related neurodevelopmental disorder wi...,NaN,MONDO:0100038,monoallelic_autosomal,NaN,strong,altered gene product structure,non_coding_transcript_variant,undetermined,inferred,NaN,NaN,NaN,40210679; 40442284,Developmental disorders,Gene now called RNU2-2. Recurrent variants: n....,2025-06-04 09:34:09+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03745


In [29]:
## saving stats on data with no gene IDs, just in case

stats_no_gene_IDs = {
    "n_rows": df[df["gene_mim"].isna() & df["hgnc_id"].isna()].shape[0],
    "n_names": len(df[df["gene_mim"].isna() & df["hgnc_id"].isna()]["gene_symbol"].unique())
}

stats_no_gene_IDs["n_rows"]
stats_no_gene_IDs["n_names"]

0

0

#### HGNC

__Running Gene HGNC IDs through NodeNorm__


Catching potential mapping failures for later stats report

In [30]:
## saving stats on data with no HGNC IDs, just in case

n_rows_no_hgnc = df["hgnc_id"].isna().sum()

In [31]:
## get set of unique CURIEs to put into NodeNorm
hgnc = df["hgnc_id"].dropna().unique()
len(hgnc)

2648

In [32]:
hgnc_nodenorm_mapping = {}

## set up variables to catch potential mapping failures
stats_hgnc_mapping_failures = {
    "unexpected_error": {},
    "nodenorm_returned_none": [],
    "wrong_category": {},
    "no_label": []
    
}

In [33]:
## larger batches are quicker
for batch in batched(hgnc, 1000):
    ## returns tuples -> cast to list
    req_body = {
        "curies": list(batch),
        "conflate": True,
    }
    r = requests.post(nodenorm_url, json=req_body)
    response = r.json()
    
    ## not doing dict comprehension. allows easier review, logic writing
    for k,v in response.items():
        ## catch unexpected errors
        try:
            ## if NodeNorm didn't have info on this ID, v will be None
            if v is not None:
                ## don't keep mapping if category is not the expected one
                if v["type"][0] == "biolink:Gene":
                    ## also throw out mapping if no primary label found
                    if v["id"].get("label"):
                        temp = {
                            k: {"primary_id": v["id"]["identifier"],
                                "primary_label": v["id"]["label"]
                               }
                        }
                        hgnc_nodenorm_mapping.update(temp)
                    else:
                        stats_hgnc_mapping_failures["no_label"].append(k)
#                         print(f"{k}: NodeNorm didn't find primary label. Not keeping this mapping.")
                else:
                    stats_hgnc_mapping_failures["wrong_category"].update({k: v["type"][0]})
#                     print(f'{k}: NodeNorm found different category {v["type"][0]}. Not keeping this mapping.')
            else:
                stats_hgnc_mapping_failures["nodenorm_returned_none"].append(k)
#                 print(f"{k}: NodeNorm didn't recognize this ID")
        except:
            stats_hgnc_mapping_failures["unexpected_error"].update({k: v})
            print(f'Encountered an unexpected error.')
            print(f'NodeNorm response key: {k}')
            print(f'NodeNorm response value: {v}')

In [34]:
len(hgnc_nodenorm_mapping)

stats_hgnc_mapping_failures

2648

{'unexpected_error': {},
 'nodenorm_returned_none': [],
 'wrong_category': {},
 'no_label': []}

#### OMIM

__Running Gene OMIM IDs through NodeNorm__

Catching potential mapping failures for later stats report. 

Pasted, adjusted from HGNC code blocks above.

In [35]:
## get set of unique CURIEs to put into NodeNorm
gene_omim = df["gene_mim"].dropna().unique()
len(gene_omim)

2647

In [36]:
gene_omim_nodenorm_mapping = {}

## set up variables to catch potential mapping failures
stats_gene_omim_mapping_failures = {
    "unexpected_error": {},
    "nodenorm_returned_none": [],
    "wrong_category": {},
    "no_label": []
    
}

In [37]:
## larger batches are quicker
for batch in batched(gene_omim, 1000):
    ## returns tuples -> cast to list
    req_body = {
        "curies": list(batch),
        "conflate": True,
    }
    r = requests.post(nodenorm_url, json=req_body)
    response = r.json()
    
    ## not doing dict comprehension. allows easier review, logic writing
    for k,v in response.items():
        ## catch unexpected errors
        try:
            ## if NodeNorm didn't have info on this ID, v will be None
            if v is not None:
                ## don't keep mapping if category is not the expected one
                if v["type"][0] == "biolink:Gene":
                    ## also throw out mapping if no primary label found
                    if v["id"].get("label"):
                        temp = {
                            k: {"primary_id": v["id"]["identifier"],
                                "primary_label": v["id"]["label"]
                               }
                        }
                        gene_omim_nodenorm_mapping.update(temp)
                    else:
                        stats_gene_omim_mapping_failures["no_label"].append(k)
#                         print(f"{k}: NodeNorm didn't find primary label. Not keeping this mapping.")
                else:
                    stats_gene_omim_mapping_failures["wrong_category"].update({k: v["type"][0]})
#                     print(f'{k}: NodeNorm found different category {v["type"][0]}. Not keeping this mapping.')
            else:
                stats_gene_omim_mapping_failures["nodenorm_returned_none"].append(k)
#                 print(f"{k}: NodeNorm didn't recognize this ID")
        except:
            stats_gene_omim_mapping_failures["unexpected_error"].update({k: v})
            print(f'Encountered an unexpected error.')
            print(f'NodeNorm response key: {k}')
            print(f'NodeNorm response value: {v}')

In [38]:
len(gene_omim_nodenorm_mapping)

stats_gene_omim_mapping_failures

2647

{'unexpected_error': {},
 'nodenorm_returned_none': [],
 'wrong_category': {},
 'no_label': []}

#### Comparing HGNC vs OMIM

In [39]:
## if row has both IDs, look for diff in mappings from each ID
for row in df[["gene_mim", "hgnc_id"]].itertuples(index=False):
    ## has both IDs
    if pd.notna(row.gene_mim) and pd.notna(row.hgnc_id):
        ## if have NodeNorm mappings for both
        if gene_omim_nodenorm_mapping.get(row.gene_mim) and \
        hgnc_nodenorm_mapping.get(row.hgnc_id):
            ## check if mappings are diff
            if gene_omim_nodenorm_mapping[row.gene_mim]["primary_id"] != \
            hgnc_nodenorm_mapping[row.hgnc_id]["primary_id"]:
                print(row)

## 2025-08-07 data: nothing prints, so there are no mismatches

In [40]:
## look for differences in name between NodeNormed and original data

for row in df[["gene_symbol", "hgnc_id"]].itertuples(index=False):
    ## works because both columns have no missing values and there's no failed mappings
    ## if this changes, need to adjust this code block
    if row.gene_symbol != hgnc_nodenorm_mapping[row.hgnc_id]["primary_label"]:
        print(f"G2P name {row.gene_symbol}, ID {row.hgnc_id}")
        print(f'NodeNorm name {hgnc_nodenorm_mapping[row.hgnc_id]["primary_label"]}, ID {hgnc_nodenorm_mapping[row.hgnc_id]["primary_id"]}')
        print("\n")

G2P name MT-TP, ID HGNC:7494
NodeNorm name TRNP, ID NCBIGene:4571


G2P name CENPJ, ID HGNC:17272
NodeNorm name CPAP, ID NCBIGene:55835


G2P name CCDC103, ID HGNC:32700
NodeNorm name DNAAF19, ID NCBIGene:388389


G2P name CCDC115, ID HGNC:28178
NodeNorm name VMA22, ID NCBIGene:84317


G2P name TMEM199, ID HGNC:18085
NodeNorm name VMA12, ID NCBIGene:147007


G2P name RNU2-2P, ID HGNC:10152
NodeNorm name RNU2-2, ID NCBIGene:26855


G2P name MT-ND1, ID HGNC:7455
NodeNorm name ND1, ID NCBIGene:4535


G2P name MT-ND4, ID HGNC:7459
NodeNorm name ND4, ID NCBIGene:4538


G2P name MT-ATP6, ID HGNC:7414
NodeNorm name ATP6, ID NCBIGene:4508


G2P name MT-ND5, ID HGNC:7461
NodeNorm name ND5, ID NCBIGene:4540


G2P name MT-ND6, ID HGNC:7462
NodeNorm name ND6, ID NCBIGene:4541




**2025-08-07 data:** 

Review of mismatched names:
* NodeNorm is correct on the official names of:
  * CENPJ -> should be CPAP
  * CCDC103 -> DNAAF19
  * CCDC115 -> VMA22
  * TMEM199 -> VMA12
  * RNU2-2P -> RNU2-2
* The rest look like mitochondrial genes, and NCBIGene main name seems to match G2P name, not NodeNorm -> messaged NodeNorm

#### Conclusions

<div class="alert alert-block alert-success">

**2025-08-07 data:** 
    
__Exploration__

* no mapping failures
* when rows have both OMIM and HGNC IDs, there were no differences in NodeNorm mapping ("mismatches")
    
__Decision: Use HGNC ID column to generate NodeNorm values__

* less missing values (none right now)
* these IDs are probably only genes (vs OMIM ID namespace has multiple kinds of entities)

### Exploring: Diseases

There are many more missing IDs for Disease, compared to Gene. 

As mentioned at the beginning of the "Pre-NodeNorming" section, I won't be using NameResolver right now. 

__This means all rows w/o any disease IDs will be removed__ because they cannot be pre-NodeNormed. 

In [41]:
df[["disease_name", "disease_mim", "disease_MONDO"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3190 entries, 0 to 3189
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   disease_name   3190 non-null   object
 1   disease_mim    2648 non-null   object
 2   disease_MONDO  1914 non-null   object
dtypes: object(3)
memory usage: 74.9+ KB


In [42]:
df[df["disease_mim"].isna() & df["disease_MONDO"].isna()]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_support,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,review,g2p_record_url
395,G2P00448,IGF2,OMIM:147470,HGNC:5466,C11ORF43; FLJ44734; IGF-II,IGF2-related Beckwith-Wiedemann syndrome,NaN,NaN,monoallelic_autosomal,imprinted region; restricted mutation set,definitive,altered gene product structure,NaN,gain of function,inferred,NaN,NaN,HP:0000006; HP:0000028; HP:0000105; HP:0000150...,NaN,Developmental disorders; Skeletal disorders,NaN,2023-05-24 09:07:28+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00448
461,G2P00522,PTHLH,OMIM:168470,HGNC:9607,HHM; PLP; PTHR; PTHRP,PTHLH-related clubbing with skeletal dysplasia...,NaN,NaN,monoallelic_autosomal,NaN,definitive,increased gene product level,NaN,undetermined,inferred,NaN,NaN,NaN,NaN,Developmental disorders,NaN,2015-07-22 16:14:37+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00522
505,G2P00569,NFIA,OMIM:600727,HGNC:7784,KIAA1439; NFI-L,NFIA-related macrocephaly with intellectual di...,NaN,NaN,monoallelic_autosomal,NaN,strong,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000023; HP:0000076; HP:0000089; HP:0000126...,17530927,Developmental disorders,NaN,2018-11-08 11:05:08+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00569
591,G2P00677,NDST1,OMIM:600853,HGNC:7680,HSST; NST1,NDST1-related intellectual disability with or ...,NaN,NaN,biallelic_autosomal,NaN,moderate,altered gene product structure,missense_variant,undetermined,inferred,NaN,NaN,NaN,21937992; 25125150; 27620904; 28211985; 286007...,Developmental disorders,NaN,2024-06-14 11:53:47+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00677
599,G2P00686,ALS2,OMIM:606352,HGNC:443,ALS2CR6,ALS2-related disorder,NaN,NaN,biallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000271; HP:0000280; HP:0000490; HP:0001249...,11586297,Developmental disorders,NaN,2025-01-21 12:35:59+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3160,G2P03718,FHOD3,OMIM:609691,HGNC:26178,FHOS2; FLJ22297; FLJ22717; KIAA1695,FHOD3-related hypertrophic cardiomyopathy,NaN,NaN,monoallelic_autosomal,NaN,definitive,altered gene product structure,inframe_deletion; inframe_insertion; missense_...,undetermined,inferred,NaN,NaN,NaN,19706596; 29907873; 30442288; 30898215; 317428...,Cardiac disorders,Pathogenic variants in FHOD3 are definitively ...,2025-08-07 09:29:49+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03718
3162,G2P03582,MYO6,OMIM:600970,HGNC:7605,DFNA22; DFNB37; KIAA0389,MYO6-related nonsyndromic genetic hearing loss,NaN,NaN,biallelic_autosomal,NaN,definitive,altered gene product structure; decreased gene...,frameshift_variant; missense_variant; splice_a...,undetermined,inferred,NaN,NaN,NaN,12687499; 18348273; 23485424; 24105371; 25999546,Ear disorders,NaN,2024-11-28 14:52:17+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03582
3163,G2P03583,MYO6,OMIM:600970,HGNC:7605,DFNA22; DFNB37; KIAA0389,MYO6-related nonsyndromic genetic hearing loss,NaN,NaN,monoallelic_autosomal,NaN,definitive,altered gene product structure; decreased gene...,frameshift_variant; missense_variant; splice_a...,undetermined,inferred,NaN,NaN,NaN,18348273; 23485424; 24105371; 25999546,Ear disorders,NaN,2024-11-28 14:47:17+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03583
3164,G2P01747,CDC14A,OMIM:603504,HGNC:1718,CDC14; CDC14A1; CDC14A2; DFNB105; DFNB32,CDC14A-related deafness,NaN,NaN,biallelic_autosomal,NaN,strong,absent gene product,NaN,loss of function,inferred,NaN,NaN,NaN,27259055,Ear disorders,NaN,2025-04-08 17:02:31+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lg

In [43]:
## saving stats on data with no disease IDs

stats_no_disease_IDs = {
    "n_rows": df[df["disease_mim"].isna() & df["disease_MONDO"].isna()].shape[0],
    "n_names": len(df[df["disease_mim"].isna() & df["disease_MONDO"].isna()]["disease_name"].unique())
}

stats_no_disease_IDs["n_rows"]
stats_no_disease_IDs["n_names"]

230

227

#### OMIM/orphanet

__Running OMIM/orphanet IDs through NodeNorm__

Catching mapping failures for later stats report

Pasted, adjusted from HGNC code blocks above.

In [44]:
## put into parser (format): DONE

## get set of unique CURIEs to put into NodeNorm
disease_OmOr = df["disease_mim"].dropna().unique()
len(disease_OmOr)

2569

In [45]:
## put into parser (format): DONE

OmOr_nodenorm_mapping = {}

## set up variables to catch mapping failures
stats_OmOr_mapping_failures = {
    "unexpected_error": {},
    "nodenorm_returned_none": [],
    "wrong_category": {},
    "no_label": []
    
}

In [46]:
## put into parser (format): DONE

## larger batches are quicker
for batch in batched(disease_OmOr, 1000):
    ## returns tuples -> cast to list
    req_body = {
        "curies": list(batch),
        "conflate": True,
    }
    r = requests.post(nodenorm_url, json=req_body)
    response = r.json()
    
    ## not doing dict comprehension. allows easier review, logic writing
    for k,v in response.items():
        ## catch unexpected errors
        try:
            ## if NodeNorm didn't have info on this ID, v will be None
            if v is not None:
                ## don't keep mapping if category is not the expected one
                if v["type"][0] == "biolink:Disease":
                    ## also throw out mapping if no primary label found
                    if v["id"].get("label"):
                        temp = {
                            k: {"primary_id": v["id"]["identifier"],
                                "primary_label": v["id"]["label"]
                               }
                        }
                        OmOr_nodenorm_mapping.update(temp)
                    else:
                        stats_OmOr_mapping_failures["no_label"].append(k)
#                         print(f"{k}: NodeNorm didn't find primary label. Not keeping this mapping.")
                else:
                    stats_OmOr_mapping_failures["wrong_category"].update({k: v["type"][0]})
#                     print(f'{k}: NodeNorm found different category {v["type"][0]}. Not keeping this mapping.')
            else:
                stats_OmOr_mapping_failures["nodenorm_returned_none"].append(k)
#                 print(f"{k}: NodeNorm didn't recognize this ID")
        except:
            stats_OmOr_mapping_failures["unexpected_error"].update({k: v})
            print(f'Encountered an unexpected error.')
            print(f'NodeNorm response key: {k}')
            print(f'NodeNorm response value: {v}')

In [47]:
## put into parser (format): DONE

## calculate stats: number of rows affected by each type of mapping failure
stats_OmOr_mapping_failures.update({
    "n_rows_none": df[df["disease_mim"].isin(stats_OmOr_mapping_failures["nodenorm_returned_none"])].shape[0],
    "n_rows_wrong_category": df[df["disease_mim"].isin(stats_OmOr_mapping_failures["wrong_category"].keys())].shape[0],
    "n_rows_no_label": df[df["disease_mim"].isin(stats_OmOr_mapping_failures["no_label"])].shape[0]
})

In [48]:
len(OmOr_nodenorm_mapping)

stats_OmOr_mapping_failures["unexpected_error"]

len(stats_OmOr_mapping_failures["nodenorm_returned_none"])
len(stats_OmOr_mapping_failures["wrong_category"])
len(stats_OmOr_mapping_failures["no_label"])

2563

{}

4

1

1

In [49]:
## code used to review mapping failures 

stats_OmOr_mapping_failures["nodenorm_returned_none"]

stats_OmOr_mapping_failures["wrong_category"]

stats_OmOr_mapping_failures["no_label"]

['OMIM:601884', 'OMIM:133701', 'OMIM:133700', 'OMIM:150800']

{'OMIM:188400': 'biolink:Gene'}

['OMIM:621034']

In [ ]:
## code used to review mapping failures 

df[df["disease_mim"] == "OMIM:133701"]

<div class="alert alert-block alert-info">

**Update 2025-09-04:**
    
4 cases where NodeNorm returned None: 
* OMIM:601884 - [valid ID](https://omim.org/entry/601884), but it doesn't seem to be a disease (previously reviewed, reported to EBI gene2pheno, NodeNorm)
* OMIM:133701 - [valid disease ID](https://omim.org/entry/133701), NodeNorm issue
* OMIM:133700 - [valid disease ID](https://omim.org/entry/133700), NodeNorm issue (previously reviewed, reported)
* OMIM:150800 - [valid disease ID](https://omim.org/entry/150800), NodeNorm issue

   
1 case where NodeNorm category was something else (currently, always Gene): 
* OMIM:188400 - [valid disease ID](https://omim.org/entry/188400), NodeNorm error (previously reviewed, reported)
    
    
1 case where NodeNorm didn't have a primary label: 
* OMIM:621034 - [valid disease ID](https://omim.org/entry/621034), NodeNorm error

<div class="alert alert-block alert-success">
    
**2025-08-07 data:** 

MONDO mappings are correct when OMIM mapping failed. 
    
But it doesn't cover all rows that have failures.

In [50]:
## code used to check how many rows have OMIM failure + MONDO ID 

df[df["disease_mim"].isin(stats_OmOr_mapping_failures["nodenorm_returned_none"]) & 
   df["disease_MONDO"].notna()]

df[df["disease_mim"].isin(stats_OmOr_mapping_failures["wrong_category"].keys()) & 
   df["disease_MONDO"].notna()]

df[df["disease_mim"].isin(stats_OmOr_mapping_failures["no_label"]) & 
   df["disease_MONDO"].notna()]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_support,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,review,g2p_record_url
427,G2P00486,EXT2,OMIM:608210,HGNC:3513,SOTV,EXT2-related multiple exostoses,OMIM:133701,MONDO:0007586,monoallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000006; HP:0000896; HP:0000918; HP:0002318...,9326317,Developmental disorders; Cancer disorders; Ske...,NaN,2023-05-24 09:07:28+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00486
643,G2P00733,EXT1,OMIM:608177,HGNC:3512,LGCR; LGS; TTV,EXT1-related multiple exostoses,OMIM:133700,MONDO:0007585,monoallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000006; HP:0000076; HP:0000252; HP:0000365...,15253765; 7550340; 8981950; 9326317,Developmental disorders; Cancer disorders; Ske...,NaN,2023-05-24 09:07:28+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00733


,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_support,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,review,g2p_record_url


,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_support,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,review,g2p_record_url


#### MONDO

__Running MONDO IDs through NodeNorm__

Catching potential mapping failures for later stats report

Pasted, adjusted from Disease OMIM/orphanet code blocks above.

In [51]:
## get set of unique CURIEs to put into NodeNorm
mondo = df["disease_MONDO"].dropna().unique()
len(mondo)

1762

In [52]:
mondo_nodenorm_mapping = {}

## set up variables to catch mapping failures
stats_mondo_mapping_failures = {
    "unexpected_error": {},
    "nodenorm_returned_none": [],
    "wrong_category": {},
    "no_label": []
    
}

In [53]:
## larger batches are quicker
for batch in batched(mondo, 1000):
    ## returns tuples -> cast to list
    req_body = {
        "curies": list(batch),
        "conflate": True,
    }
    r = requests.post(nodenorm_url, json=req_body)
    response = r.json()
    
    ## not doing dict comprehension. allows easier review, logic writing
    for k,v in response.items():
        ## catch unexpected errors
        try:
            ## if NodeNorm didn't have info on this ID, v will be None
            if v is not None:
                ## don't keep mapping if category is not the expected one
                if v["type"][0] == "biolink:Disease":
                    ## also throw out mapping if no primary label found
                    if v["id"].get("label"):
                        temp = {
                            k: {"primary_id": v["id"]["identifier"],
                                "primary_label": v["id"]["label"]
                               }
                        }
                        mondo_nodenorm_mapping.update(temp)
                    else:
                        stats_mondo_mapping_failures["no_label"].append(k)
#                         print(f"{k}: NodeNorm didn't find primary label. Not keeping this mapping.")
                else:
                    stats_mondo_mapping_failures["wrong_category"].update({k: v["type"][0]})
#                     print(f'{k}: NodeNorm found different category {v["type"][0]}. Not keeping this mapping.')
            else:
                stats_mondo_mapping_failures["nodenorm_returned_none"].append(k)
#                 print(f"{k}: NodeNorm didn't recognize this ID")
        except:
            stats_mondo_mapping_failures["unexpected_error"].update({k: v})
            print(f'Encountered an unexpected error.')
            print(f'NodeNorm response key: {k}')
            print(f'NodeNorm response value: {v}')

In [54]:
## put into parser (format): 

## calculate stats: number of rows affected by each type of mapping failure
stats_mondo_mapping_failures.update({
    "n_rows_none": df[df["disease_MONDO"].isin(stats_mondo_mapping_failures["nodenorm_returned_none"])].shape[0],
    "n_rows_wrong_category": df[df["disease_MONDO"].isin(stats_mondo_mapping_failures["wrong_category"].keys())].shape[0],
    "n_rows_no_label": df[df["disease_MONDO"].isin(stats_mondo_mapping_failures["no_label"])].shape[0]
})

In [55]:
len(mondo_nodenorm_mapping)

stats_mondo_mapping_failures

1760

{'unexpected_error': {},
 'nodenorm_returned_none': ['MONDO:0976127', 'MONDO:0976124'],
 'wrong_category': {},
 'no_label': [],
 'n_rows_none': 2,
 'n_rows_wrong_category': 0,
 'n_rows_no_label': 0}

**2025-09-04:**

* MONDO:0976127 - [valid disease ID](https://monarchinitiative.org/MONDO:0976127), NodeNorm issue
* MONDO:0976124 - [valid disease ID](https://monarchinitiative.org/MONDO:0976124), NodeNorm issue

In [56]:
df[df["disease_MONDO"] == "MONDO:0976127"]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_support,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,review,g2p_record_url
2200,G2P03231,HYAL2,OMIM:603551,HGNC:5321,LUCA-2; LUCA2,HYAL2-related syndrome with cleft lip and pala...,NaN,MONDO:0976127,biallelic_autosomal,NaN,strong,absent gene product,NaN,undetermined,inferred,NaN,NaN,NaN,28081210; 34906488,Developmental disorders,NaN,2022-01-17 11:11:48+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03231


#### Comparing OMIM/orphanet vs MONDO

In [57]:


## if row has both IDs, look for diff in mappings from each ID

## list of tuples (omim/orpha, mondo)
mismatches = []

for row in df[["disease_mim", "disease_MONDO"]].itertuples(index=False):
    ## has both IDs
    if pd.notna(row.disease_mim) and pd.notna(row.disease_MONDO):
        ## if have NodeNorm mappings for both
        if OmOr_nodenorm_mapping.get(row.disease_mim) and \
        mondo_nodenorm_mapping.get(row.disease_MONDO):
            ## check if mappings are diff
            if OmOr_nodenorm_mapping[row.disease_mim]["primary_id"] != \
            mondo_nodenorm_mapping[row.disease_MONDO]["primary_id"]:
                mismatches.append((row.disease_mim, row.disease_MONDO))

print(f"There's {len(mismatches)} mismatches between the OMIM/orphanet and MONDO NodeNorm mappings.")

There's 45 mismatches between the OMIM/orphanet and MONDO NodeNorm mappings.


In [58]:
## code chunk to review mismatches 1 by 1
mismatches[12]

('OMIM:610738', 'MONDO:0012548')

In [59]:
## code chunk to review mismatches 1 by 1

OmOr_nodenorm_mapping["OMIM:610738"]
mondo_nodenorm_mapping["MONDO:0012548"]

{'primary_id': 'OMIM:610738',
 'primary_label': 'Neutropenia, Severe Congenital, Autosomal Recessive 3'}

{'primary_id': 'MONDO:0012548', 'primary_label': 'Kostmann syndrome'}

In [60]:
## code chunk to review mismatches 1 by 1

df[df["disease_mim"] == "OMIM:610738"]
df[df["disease_MONDO"] == "MONDO:0012548"]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_support,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,review,g2p_record_url
948,G2P01078,HAX1,OMIM:605998,HGNC:16915,HAX-1; HCLSBP1; HS1BP1,"HAX1-related neutropenia, severe congenital",OMIM:610738,MONDO:0012548,biallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000007; HP:0001250; HP:0001263; HP:0001875...,17187068; 18337561; 18611981; 19036076,Developmental disorders,NaN,2015-07-22 16:15:09+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P01078


,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_support,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,review,g2p_record_url
948,G2P01078,HAX1,OMIM:605998,HGNC:16915,HAX-1; HCLSBP1; HS1BP1,"HAX1-related neutropenia, severe congenital",OMIM:610738,MONDO:0012548,biallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000007; HP:0001250; HP:0001263; HP:0001875...,17187068; 18337561; 18611981; 19036076,Developmental disorders,NaN,2015-07-22 16:15:09+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P01078


<div class="alert alert-block alert-info">    

**2025-08-07 data:** 

__Review of OMIM vs MONDO NodeNorm mismatches__ (45 total, reviewed 23)

None were orphanet.    
**Looked over the pairs that were reviewed last time** (2025-02-28) and more

---

__10: OMIM's mapping is better__

> __2: Mondo ID is related but wrong__ -> emailed EBI gene2pheno w/ example
> * 'OMIM:101000', 'MONDO:0008075': omim is correct type of schwannomatosis (NF2/type 2), vs mondo is a sibling (both under neurofibromatosis MONDO:0021061)
>   * NodeNorm should map omim to MONDO:0007039 but isn't -> messaged NodeNorm
> * 'OMIM:613987', __'MONDO:0009136'__: omim is correct recessive 2, but mondo is recessive 1 (diff gene? Confusing because Monarch page links to gene NHP2 but OMIM page doesn't)
>   * NodeNorm should map omim to MONDO:0013519 but isn't -> messaged NodeNorm  

> __8: Mondo ID is too general__ (can see on Monarch website) -> emailed EBI gene2pheno w/ example
> * 'OMIM:300696', 'MONDO:0010680': omim maps to MONDO:0010401, child of the mondo
> * 'OMIM:304120', 'MONDO:0019027': omim maps to MONDO:0010571 (syndrome type 2), child of the mondo (syndrome)
> * 'OMIM:610019', 'MONDO:0005129': omim maps to MONDO:0012395 (cataract 18), child of the mondo (cataract)
> * 'OMIM:611726', 'MONDO:0016295': omim maps to MONDO:0012721, child of the mondo 
> * 'OMIM:602668', 'MONDO:0016107': omim maps to MONDO:0011266 (type 2), child of the mondo
> * 'OMIM:203200', 'MONDO:0018910': omim maps to MONDO:0008746 (type 2), child of the mondo
> * 'OMIM:614328', 'MONDO:0017411': omim maps to MONDO:0013693 (type 1), child of the mondo
> * 'OMIM:175800', 'MONDO:0006602': omim maps to MONDO:0008290 (1, mibelli type), grandchild of the mondo



**1: MONDO's mapping is better**
* 'OMIM:613723', 'MONDO:0009181': mondo matches the disease name and phenotypes listed in the record better than the omim 


**2: Unsure**
* 'OMIM:614886', 'MONDO:0100270': both are technically correct. OMIM maps to more specific MONDO:0013951 (only child of the mondo provided) 
* 'OMIM:612463', 'MONDO:0019992': the records involved (G2P00591, G2P02117) are for gene GNAS. This gene is tied to both pseudopseudohypoparathyroidism AND specific subtypes of pseudohypoparathyroidism. Both records seem to have somewhat incorrect ID assignments. 


**10: NodeNorm error**
* 'OMIM:224230', 'MONDO:0009136': both are recessive 1, NodeNorm should map to same entity  -> messaged NodeNorm
* 'OMIM:613988', 'MONDO:0013520': both are autosomal recessive 3, NodeNorm should map to same entity -> messaged NodeNorm
* 'OMIM:616353', 'MONDO:0009136': both are recessive 6, NodeNorm should map to same entity -> messaged NodeNorm
* 'OMIM:257300', 'MONDO:0009759': both are same entity according to Monarch 
* 'OMIM:609322', 'MONDO:0012252': both are same entity according to Monarch 
* 'OMIM:162200', 'MONDO:0018975': both are same entity according to Monarch 
* 'OMIM:162300', 'MONDO:0008082': both are same entity according to Monarch
* 'OMIM:240500', 'MONDO:0009413': both are same entity according to Monarch
* 'OMIM:614114', 'MONDO:0013582': both are same entity according to Monarch
* 'OMIM:610738', 'MONDO:0012548': both are same entity according to Monarch

<div class="alert alert-block alert-success">    

**CONCLUSION**
    
**2025-08-07 data:** 

Prefer the disease_mim column (OMIM/orphanet IDs) to MONDO, if a row has both

#### Checking MONDO data

Above, I decided the OMIM/orphanet disease IDs were better. 

However, I wondered if the MONDO IDs were accurate to the disease name when there weren't OMIM/orphanet IDs. Then they could be used for NodeNorming and less data would be dropped because there weren't IDs for NodeNorming.

In [61]:
## get the data that has MONDO, doesn't have OMIM/orphanet

df_mondo_only = df[df["disease_mim"].isna() & df["disease_MONDO"].notna()].copy()

mondo_only = df_mondo_only["disease_MONDO"].dropna().unique()

In [62]:
## saving stats on data with only MONDO ID

stats_mondo_only = {
    "n_rows": df_mondo_only.shape[0],
    "n_names": len(mondo_only)
}

stats_mondo_only["n_rows"]
stats_mondo_only["n_names"]

312

188

In [224]:
## code chunk used to review some of the data

# pd.set_option('display.max_colwidth', None)
# pd.reset_option('display.max_colwidth')

# df_mondo_only[df_mondo_only["disease_MONDO"] == mondo_only[4]]
# df[df["disease_MONDO"] == "MONDO:0010912"]

# df[df["disease_name"] == "DSC2-related arrhythmogenic right ventricular cardiomyopathy"]
# df[df["gene_symbol"] == "ALPL"]

# df_mondo_only[df_mondo_only["panel"].str.contains("Ear")]

<div class="alert alert-block alert-info">    

**2025-08-07 data:** 

__Reviewed some of the data__

Method: look at rows (aka MONDO ID/disease name pairs). **Looked over the pairs that were reviewed last time** (2025-02-28) and more. Some weren't in the data (perhaps filtered out earlier, removed from dataset, or now had omim + mondo IDs in row).

    
__Summary__
* 30 rows / 296 (~10%)
* None were wrong! 
* Could tell EBI gene2pheno of issues but they are similar to those listed in mismatch mapping section

__Details__

__2 MONDO is too general__ 
* "MONDO:0018965" (Alport syndrome) for "COL4A5-related Alport syndrome". The COL4A5-specific version is a child term: MONDO:0010520/OMIM:301050  (X-linked)
* "MONDO:0014980" (cone-rod dystrophy and hearing loss) for "CEP78-related cone-rod dystrophy and hearing loss". The CEP78-specific version is a child term: MONDO:0020778/OMIM:617236


__2 Unsure -> TELL EBI GENE2PHENO?__
* "MONDO:0005129" for "CYP51A1-related congenital cataract": mondo is cataract, which is not wrong but kinda general. MONDO:0033853 seems better (correlated with gene, matches phenotypes, orphanet ref uses one of the ref papers) 
* "MONDO:0020367" for "MYOC-related juvenile open angle glaucoma": while mondo is almost-exact name match, it's not directly linked to gene. Instead, its child disease is directly linked to gene MONDO:0007664/OMIM:137750 (glaucoma 1, open angle, A) 


__11 Okay (using general term is fine)__
* **{counts as two}** "MONDO:0024676" (childhood kidney Wilms tumor) for "CTR9-related Wilms tumour", "TRIM28-": couldn't find better mapping. TRIM28 is correlated to parent term (kidney Wilms tumor). 
* "MONDO:0019118" for "CLN3-related retinal dystrophy": couldn't find better mapping 
* "MONDO:0018570" for "ALPL-related hypophosphatasia": there are more specific terms, but it's unclear which one to use here. So using more general one is fine
* "MONDO:0015469" for "PRRX1-related craniosynostosis": couldn't find better mapping 
* "MONDO:0018094" for "SNAI2-related Waardenburg syndrome": while MONDO maps to the more specific type 2 (MONDO:0019517), this specific type's inheritance pattern doesn't match (dominant/monoallelic vs recessive/biallelic)
* "MONDO:0100124" for "NAA10-related nonpecific severe intellectual disability": there are more specific terms, but it's unclear which one to use here. So using more general one is fine
* **{counts as two}** "MONDO:0001071" for "SCN2A-related nonspecific severe intellectual disability", "AFF3-related intellectual disability": there are more specific terms linked to the gene, but it's unclear which one to use here. So using more general one is fine
* "MONDO:0100062" for "FLNA-related epileptic encephalopathy": there are more specific terms linked to the gene, but it's unclear which one to use here. So using more general one is fine
* "MONDO:0700092" for "PRRT2-related intellectual developmental disorder": there are more specific terms linked to the gene, but it's unclear which one to use here. So using more general one is fine


__15 Great__
* "MONDO:0012506" for "DSC2-related arrhythmogenic right ventricular cardiomyopathy"
* "MONDO:0011001" for "SCN5A-related Brugada syndrome"
* "MONDO:0013262" for "MYH7-related dilated cardiomyopathy"
* "MONDO:0013369" for "TNNI3-related hypertrophic cardiomyopathy"
* "MONDO:0010946" for "PRKAG2-related cardiomyopathy"
* "MONDO:0014143" for "RIT1-related Noonan syndrome"
* "MONDO:0014214" for "DYNC2I1-related short-rib polydactyly"
* "MONDO:0013522" for "TINF2-related dyskeratosis congenita"
* "MONDO:0010215" for "ERCC4-related xeroderma pigmentosum, group F"
* "MONDO:0009735" for "SPINK5-related Netherton syndrome"
* "MONDO:0007566" for "TGFBR1-related multiple self-healing squamous epithelioma"
* "MONDO:0007485" for "TERC-related dyskeratosis congenita"
* "MONDO:0032835" for "MIR140-related spondyloepiphyseal dysplasia, Nishimura type"
* "MONDO:0001071" for "GSPT2-related intellectual disability"
* "MONDO:0009869" for "SOX9-related Pierre Robin sequence"

#### Conclusions

<div class="alert alert-block alert-success">

**2025-08-07 data:** 
    
__Exploration__

* some rows have no disease IDs -> will filter these out (can't NodeNorm)
* very few NodeNorm mapping failures for IDs
* when rows have both OMIM and MONDO ID, use OMIM ID - generally more accurate
  * while it would be nice to use MONDO ID if NodeNorm fails for OMIM ID, the common ingest pipeline won't have this level of sophistication to do this
* when row only has MONDO ID, can use it - accurate enough now!
    
__Decision: Use OMIM ID column as primary, MONDO ID if don't have OMIM ID__

## Stats on rows removed during NodeNorming

This section prints the statistics on rows in the original data that were removed. 

(Uses variables generated during the previous section "Pre-NodeNorming")

<div class="alert alert-block alert-success">

**2025-08-07 data:** 

Genes: No rows removed due to lack of IDs for NodeNorming or NodeNorm mapping issues.

In [63]:
## partial put into parser (format): DONE

print("Gene Pre-NodeNorming\n")

## no gene IDs
print(f'{stats_no_gene_IDs["n_rows"]} row(s) with no gene IDs')

## no HGNC IDs: key column for NodeNorming
print(f'{n_rows_no_hgnc} row(s) with no HGNC IDs')

## HGNC NodeNorm issues: none, but showing anyways
print("\n")
print("HGNC NodeNorm mapping failures:")

print(f'IDs with no data in NodeNorm: {len(stats_hgnc_mapping_failures["nodenorm_returned_none"])}')
print(f'IDs with the wrong NodeNormed category: {len(stats_hgnc_mapping_failures["wrong_category"])}')
print(f'IDs with no label in NodeNorm: {len(stats_hgnc_mapping_failures["no_label"])}')

Gene Pre-NodeNorming

0 row(s) with no gene IDs
0 row(s) with no HGNC IDs


HGNC NodeNorm mapping failures:
IDs with no data in NodeNorm: 0
IDs with the wrong NodeNormed category: 0
IDs with no label in NodeNorm: 0


<div class="alert alert-block alert-success">

**2025-08-07 data:** 
    
__Diseases: some rows removed__ due to lack of IDs for NodeNorming. A few rows removed due NodeNorm mapping issues.

In [64]:
stats_OmOr_mapping_failures
stats_mondo_mapping_failures

{'unexpected_error': {},
 'nodenorm_returned_none': ['OMIM:601884',
  'OMIM:133701',
  'OMIM:133700',
  'OMIM:150800'],
 'wrong_category': {'OMIM:188400': 'biolink:Gene'},
 'no_label': ['OMIM:621034'],
 'n_rows_none': 4,
 'n_rows_wrong_category': 1,
 'n_rows_no_label': 1}

{'unexpected_error': {},
 'nodenorm_returned_none': ['MONDO:0976127', 'MONDO:0976124'],
 'wrong_category': {},
 'no_label': [],
 'n_rows_none': 2,
 'n_rows_wrong_category': 0,
 'n_rows_no_label': 0}

In [65]:
## partial put into parser (format): 

print("Disease Pre-NodeNorming\n")

## no disease IDs
print(f'{stats_no_disease_IDs["n_rows"]} row(s) with no disease IDs '
      f'(= {stats_no_disease_IDs["n_names"]} unique diseases)')

## OMIM NodeNorm issues
print("\n")
print("OMIM NodeNorm mapping failures:")

print(f'{stats_OmOr_mapping_failures["n_rows_none"]} row(s) for '
      f'{len(stats_OmOr_mapping_failures["nodenorm_returned_none"])} '
      f'IDs with no data in NodeNorm')

print(f'{stats_OmOr_mapping_failures["n_rows_wrong_category"]} row(s) for '
      f'{len(stats_OmOr_mapping_failures["wrong_category"])} '
      f'IDs with the wrong NodeNormed category')

print(f'{stats_OmOr_mapping_failures["n_rows_no_label"]} row(s) for '
      f'{len(stats_OmOr_mapping_failures["no_label"])} '
      f'IDs with no label in NodeNorm')

## MONDO NodeNorm issues
print("\n")
print("MONDO NodeNorm mapping failures:")

print(f'{stats_mondo_mapping_failures["n_rows_none"]} row(s) for '
      f'{len(stats_mondo_mapping_failures["nodenorm_returned_none"])} '
      f'IDs with no data in NodeNorm')

print(f'{stats_mondo_mapping_failures["n_rows_wrong_category"]} row(s) for '
      f'{len(stats_mondo_mapping_failures["wrong_category"])} '
      f'IDs with the wrong NodeNormed category')

print(f'{stats_mondo_mapping_failures["n_rows_no_label"]} row(s) for '
      f'{len(stats_mondo_mapping_failures["no_label"])} '
      f'IDs with no label in NodeNorm')

Disease Pre-NodeNorming

230 row(s) with no disease IDs (= 227 unique diseases)


OMIM NodeNorm mapping failures:
4 row(s) for 4 IDs with no data in NodeNorm
1 row(s) for 1 IDs with the wrong NodeNormed category
1 row(s) for 1 IDs with no label in NodeNorm


MONDO NodeNorm mapping failures:
2 row(s) for 2 IDs with no data in NodeNorm
0 row(s) for 0 IDs with the wrong NodeNormed category
0 row(s) for 0 IDs with no label in NodeNorm


<div class="alert alert-block alert-success">
    
__Totals__

In [66]:
## put into parser (format): DONE

n_rows_before_nodenorm = df.shape[0]
n_rows_nodenorm_removed = stats_no_disease_IDs["n_rows"] + \
                          stats_OmOr_mapping_failures["n_rows_none"] + \
                          stats_OmOr_mapping_failures["n_rows_wrong_category"] + \
                          stats_OmOr_mapping_failures["n_rows_no_label"] + \
                          stats_mondo_mapping_failures["n_rows_none"] + \
                          stats_mondo_mapping_failures["n_rows_wrong_category"] + \
                          stats_mondo_mapping_failures["n_rows_no_label"]


n_rows_after_nodenorm = n_rows_before_nodenorm - n_rows_nodenorm_removed

print(f"{n_rows_before_nodenorm} rows/records before Pre-NodeNorming\n")

print(f"Estimated {n_rows_nodenorm_removed} rows removed during Disease NodeNorming process\n")

print(f"Estimated {n_rows_after_nodenorm} rows/records left ({n_rows_after_nodenorm/n_rows_before_nodenorm:.1%})")

3190 rows/records before Pre-NodeNorming

Estimated 238 rows removed during Disease NodeNorming process

Estimated 2952 rows/records left (92.5%)


This is estimated because the OMIM/MONDO NodeNorm failures are assumed to affect diff rows, but they could overlap if a row had both IDs and they both failed

## Adding NodeNorm data, removing rows

Using gene HGNC and disease OMIM/orphanet or MONDO IDs for pre-NodeNorming

In [67]:
df.shape

(3190, 24)

In [68]:
## create single column to use for disease NodeNorming
df["disease_original_id"] = [
    x.disease_mim if pd.notna(x.disease_mim)
    else x.disease_MONDO if pd.notna(x.disease_MONDO)
    else pd.NA
    for x in df[["disease_mim", "disease_MONDO"]].itertuples()
]

In [69]:
## double-check new column logic

## should be no rows where original ID is MONDO but there's OMIM ID
df[df["disease_original_id"].str.contains("MONDO", na=False) &
   df["disease_mim"].notna()
  ]

## counts: should equal df shape
count1 = df[df["disease_original_id"].str.contains("OMIM", na=False)].shape[0]
count2 = df[df["disease_original_id"].str.contains("orphanet", na=False)].shape[0]
count3 = df[df["disease_original_id"].str.contains("MONDO", na=False)].shape[0]
count4 = df[df["disease_original_id"].isna()].shape[0]

print(f"OMIM rows: {count1}")
print(f"orphanet rows: {count2}")
print(f"MONDO rows: {count3}")
print(f"no ID rows: {count4}")

df.shape[0] == count1 + count2 + count3 + count4

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_support,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,review,g2p_record_url,disease_original_id


OMIM rows: 2647
orphanet rows: 1
MONDO rows: 312
no ID rows: 230


True

In [233]:
## another idea for how to create column
## NOT EQUIVALENT, will use MONDO to fix OMIM failures

# look = [
#     OmOr_nodenorm_mapping[x.disease_mim]["primary_id"] if OmOr_nodenorm_mapping.get(x.disease_mim) 
#     else mondo_nodenorm_mapping[x.disease_MONDO]["primary_id"] if mondo_nodenorm_mapping.get(x.disease_MONDO)
#     else pd.NA
#     for x in df[["disease_mim", "disease_MONDO"]].itertuples()]

# ## this is a crude way to check if OMIM IDs were used - they may resolve to non-MONDO primary IDs
# for i in look:
#     if pd.notna(i):
#         if "DOID" in i:
#             print(i)

In [70]:
## put into parser (format): 

## Gene: assumes no missing values
df["gene_nodenorm_id"] = [hgnc_nodenorm_mapping[i]["primary_id"] for i in df["hgnc_id"]]
df["gene_nodenorm_label"] = [hgnc_nodenorm_mapping[i]["primary_label"] for i in df["hgnc_id"]]

df["disease_nodenorm_id"] = [
    ## should work for OMIM and orphanet
    OmOr_nodenorm_mapping[i]["primary_id"] if OmOr_nodenorm_mapping.get(i)
    else mondo_nodenorm_mapping[i]["primary_id"] if mondo_nodenorm_mapping.get(i)
    else pd.NA
    for i in df["disease_original_id"]
]

df["disease_nodenorm_label"] = [
    ## should work for OMIM and orphanet
    OmOr_nodenorm_mapping[i]["primary_label"] if OmOr_nodenorm_mapping.get(i)
    else mondo_nodenorm_mapping[i]["primary_label"] if mondo_nodenorm_mapping.get(i)
    else pd.NA
    for i in df["disease_original_id"]
]

In [71]:
## move disease_original_id to end for easier comparison

df["disease_original_id"] = df.pop("disease_original_id")

In [72]:
df.head()

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_support,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,review,g2p_record_url,gene_nodenorm_id,gene_nodenorm_label,disease_nodenorm_id,disease_nodenorm_label,disease_original_id
0,G2P00001,HMX1,OMIM:142992,HGNC:5017,H6; NKX5-3,HMX1-related oculoauricular syndrome,OMIM:612109,MONDO:0012802,biallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000007; HP:0000482; HP:0000505; HP:0000518...,18423520; 25574057; 29140751,Developmental disorders; Eye disorders,NaN,2019-09-26 16:23:46+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00001,NCBIGene:3166,HMX1,MONDO:0012802,oculoauricular syndrome,OMIM:612109
1,G2P00002,SLX4,OMIM:613278,HGNC:23845,BTBD12; FANCP; KIAA1784; KIAA1987,SLX4-related Fanconi anemia,OMIM:613951,MONDO:0013499,biallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000007; HP:0000028; HP:0000085; HP:0000125...,21240275; 21240277,Developmental disorders,NaN,2025-01-28 23:09:54+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00002,NCBIGene:84464,SLX4,MONDO:0013499,Fanconi anemia complementation group P,OMIM:613951
2,G2P00003,ARG1,OMIM:608313,HGNC:663,NaN,ARG1-related argininemia,OMIM:207800,MONDO:0008814,biallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000007; HP:0000737; HP:0000752; HP:0001249...,10502833; 1463019; 1598908; 2365823; 7649538,Developmental disorders,NaN,2015-07-22 16:14:07+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00003,NCBIGene:383,ARG1,MONDO:0008814,Argininemia,OMIM:207800
3,G2P00004,ATR,OMIM:601215,HGNC:882,FRP1; MEC1; SCKL; SCKL1,ATR-related Seckel syndrome,OMIM:210600,MONDO:0008869,biallelic_autosomal,NaN,strong,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000007; HP:0000028; HP:0000047; HP:0000057...,NaN,Developmental disorders; Skeletal disorders,NaN,2025-01-27 14:24:27+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00004,NCBIGene:545,ATR,MONDO:0008869,Seckel syndrome 1,OMIM:210600
4,G2P00005,FANCB,OMIM:300515,HGNC:3583,FAAP95; FAB; FLJ34064,FANCB-related Fanconi anemia,OMIM:300514,MONDO:0010351,monoallelic_X_hemizygous,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000083; HP:0000100; HP:0000119; HP:0000707...,15502827; 16679491; 21910217; 22052692; 236135...,Developmental disorders; Skin disorders,NaN,2024-08-20 14:13:58+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00005,NCBIGene:2187,FANCB,MONDO:0010351,Fanconi anemia complementation group B,OMIM:300514


In [73]:
## double-check disease nodenorm logic: 

df[df["disease_mim"].isna()].head()

df[df["disease_mim"].isna() & df["disease_MONDO"].isna()].head()

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_support,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,review,g2p_record_url,gene_nodenorm_id,gene_nodenorm_label,disease_nodenorm_id,disease_nodenorm_label,disease_original_id
14,G2P00016,NAA10,OMIM:300013,HGNC:18704,ARD1; ARD1A; DXS707; TE2,NAA10-related nonpecific severe intellectual d...,NaN,MONDO:0100124,monoallelic_X_heterozygous,NaN,definitive,altered gene product structure,inframe_deletion; inframe_insertion; missense_...,undetermined,inferred,NaN,NaN,NaN,25099252,Developmental disorders,NaN,2015-07-22 16:14:09+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00016,NCBIGene:8260,NAA10,MONDO:0100124,NAA10-related syndrome,MONDO:0100124
30,G2P00033,SCN2A,OMIM:182390,HGNC:10588,HBSCI; HBSCII; NAV1.2; SCN2A1; SCN2A2,SCN2A-related nonspecific severe intellectual ...,NaN,MONDO:0001071,monoallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,NaN,30062040,Developmental disorders,NaN,2015-07-22 16:14:11+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00033,NCBIGene:6326,SCN2A,MONDO:0001071,intellectual disability,MONDO:0001071
52,G2P00057,FLNA,OMIM:300017,HGNC:3754,ABP-280; FLN; FLN1; OPD1; OPD2,FLNA-related epileptic encephalopathy,NaN,MONDO:0100062,monoallelic_X_heterozygous,NaN,definitive,altered gene product structure,inframe_deletion; inframe_insertion; missense_...,undetermined,inferred,NaN,NaN,NaN,23934111,Developmental disorders; Skin disorders,NaN,2025-01-27 22:44:48+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00057,NCBIGene:2316,FLNA,MONDO:0100062,developmental and epileptic encephalopathy,MONDO:0100062
63,G2P00068,SOX9,OMIM:608160,HGNC:11204,CMD1; CMPD1; SRA1,SOX9-related Pierre Robin sequence,NaN,MONDO:0009869,monoallelic_autosomal,NaN,definitive,NaN,regulatory_region_variant,undetermined,inferred,NaN,NaN,NaN,NaN,Developmental disorders,NaN,2015-07-22 16:14:14+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00068,NCBIGene:6662,SOX9,MONDO:0009869,isolated Pierre-Robin syndrome,MONDO:0009869
78,G2P00086,PRRT2,OMIM:614386,HGNC:30500,DKFZP547J199; DSPB3; DYT10; EKD1; FICCA; FLJ25...,PRRT2-related intellectual developmental disorder,NaN,MONDO:0700092,biallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,NaN,21937992,Developmental disorders,NaN,2015-07-22 16:14:14+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00086,NCBIGene:112476,PRRT2,MONDO:0700092,neurodevelopmental disorder,MONDO:0700092


,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_support,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,review,g2p_record_url,gene_nodenorm_id,gene_nodenorm_label,disease_nodenorm_id,disease_nodenorm_label,disease_original_id
395,G2P00448,IGF2,OMIM:147470,HGNC:5466,C11ORF43; FLJ44734; IGF-II,IGF2-related Beckwith-Wiedemann syndrome,NaN,NaN,monoallelic_autosomal,imprinted region; restricted mutation set,definitive,altered gene product structure,NaN,gain of function,inferred,NaN,NaN,HP:0000006; HP:0000028; HP:0000105; HP:0000150...,NaN,Developmental disorders; Skeletal disorders,NaN,2023-05-24 09:07:28+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00448,NCBIGene:3481,IGF2,<NA>,<NA>,<NA>
461,G2P00522,PTHLH,OMIM:168470,HGNC:9607,HHM; PLP; PTHR; PTHRP,PTHLH-related clubbing with skeletal dysplasia...,NaN,NaN,monoallelic_autosomal,NaN,definitive,increased gene product level,NaN,undetermined,inferred,NaN,NaN,NaN,NaN,Developmental disorders,NaN,2015-07-22 16:14:37+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00522,NCBIGene:5744,PTHLH,<NA>,<NA>,<NA>
505,G2P00569,NFIA,OMIM:600727,HGNC:7784,KIAA1439; NFI-L,NFIA-related macrocephaly with intellectual di...,NaN,NaN,monoallelic_autosomal,NaN,strong,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000023; HP:0000076; HP:0000089; HP:0000126...,17530927,Developmental disorders,NaN,2018-11-08 11:05:08+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00569,NCBIGene:4774,NFIA,<NA>,<NA>,<NA>
591,G2P00677,NDST1,OMIM:600853,HGNC:7680,HSST; NST1,NDST1-related intellectual disability with or ...,NaN,NaN,biallelic_autosomal,NaN,moderate,altered gene product structure,missense_variant,undetermined,inferred,NaN,NaN,NaN,21937992; 25125150; 27620904; 28211985; 286007...,Developmental disorders,NaN,2024-06-14 11:53:47+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00677,NCBIGene:3340,NDST1,<NA>,<NA>,<NA>
599,G2P00686,ALS2,OMIM:606352,HGNC:443,ALS2CR6,ALS2-related disorder,NaN,NaN,biallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,NaN,HP:0000271; HP:0000280; HP:0000490; HP:0001249...,11586297,Developmental disorders,NaN,2025-01-21 12:35:59+00:00,NaN,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00686,NCBIGene:57679,ALS2,<NA>,<NA>,<NA>


In [74]:
## put into parser (change in-place): DONE

df_only_nodenormed = df.dropna(subset=["gene_nodenorm_id", "gene_nodenorm_label", 
                                       "disease_nodenorm_id", "disease_nodenorm_label", 
                                       "disease_original_id"],
                              ignore_index=True).copy()

In [75]:
## same! so it works as expected

df_only_nodenormed.shape

n_rows_after_nodenorm

(2952, 29)

2952

In [76]:
df_only_nodenormed.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 29 columns):
 #   Column                              Non-Null Count  Dtype              
---  ------                              --------------  -----              
 0   g2p_id                              2952 non-null   object             
 1   gene_symbol                         2952 non-null   object             
 2   gene_mim                            2951 non-null   object             
 3   hgnc_id                             2952 non-null   object             
 4   previous_gene_symbols               2730 non-null   object             
 5   disease_name                        2952 non-null   object             
 6   disease_mim                         2642 non-null   object             
 7   disease_MONDO                       1910 non-null   object             
 8   allelic_requirement                 2952 non-null   object             
 9   cross_cutting_modifier              375 n

In [ ]:
## code block to review NodeNorm "duplicate edges" - from DISEASES

# df_textmining_nodenorm_dups = df_textmining[
#     df_textmining.duplicated(
#         subset=["gene_nodenorm_id", "disease_nodenorm_id"], 
#         keep=False)].copy()

# df_textmining_nodenorm_dups.sort_values(by=["disease_nodenorm_id", "gene_nodenorm_id"],
#                                         inplace=True)

# df_textmining_nodenorm_dups.shape

# df_textmining_nodenorm_dups

## Generating documents

### Rows not included

<div class="alert alert-block alert-info">

See section "Stats on rows removed during NodeNorming"
* No IDs in both disease_mim and disease_MONDO columns
* NodeNorm mapping failures for disease_mim column IDs or for disease_MONDO column IDs (only relevant if disease_MONDO ID was used for NodeNorming)
* confidence column value == ("refuted" OR "disputed" OR "limited")

### Columns not included

<div class="alert alert-block alert-info">

See data-playground for details

<br>

Seem **easier** to get into Translator, potentially useful: 

- **confidence**: 
   - there's biolink association-slot *has confidence level*. But there's also a biolink entity *confidence level* that's supposed to have values from CIO. 
   - Are G2P's terms okay? Or are they supposed to be mapped to ontology terms like CIO/SEPIO?-(which...may be a loss of info compared to G2P's term definitions)

<br>

Seem harder to get into Translator, potentially useful: 

- **cross_cutting_modifier**: additional info on inheritance. Limited set of terms BUT "; "- delimited. Some terms may map to "HPO inheritance qualifier terms" (didn't try). Lots of missing data (NA). 
  - would be a new edge/node property or qualifier. But complicated because EBI gene2pheno has custom terms, not just from HPO inheritance qualifiers. 
- **variant_consequence**: row can have multiple values ("; "- delimited). Limited set of terms already mapped to SO.
  - seems like aspect qualifier, but this can be a list for a gene-disease edge - and I'm not sure how to handle this (not that comfortable splitting into multiple edges)
- **variant_types**: row can have multiple values ("; "- delimited). Medium set of terms already mapped to SO. Lots of missing data (NA). **>100 UNIQUE VALUES (SINGLE, MIX OF TERMS). I don't think using this as a qualifier is a good idea - it's confusing**
  - would be a new edge/node property or qualifier (somewhat modeled as predicates, for variant-gene relationships).
- **molecular_mechanism_support** (**new Aug 2025**): "qualifies" the molecular_mechanism (seems to say how molecular mechanism was decided: "inferred" or "evidence"). 
  - **Doesn't seem to show up on single-record webpage** (compared rows with value "evidence" / no "molecular_mechanism_categorisation" value to their webpages; the webpages don't have the string "evidence").
  - tricky since it's like "how knowledge was obtained" for a specific part of edge (I'm using molecular_mechanism to adjust the subject qualifier) 
- **molecular mechanism categorisation** (**new Aug 2025**): more detailed mechanism info, structured (displayed in single-record webpage as a table). 
  - VERY SPARSE (LOTS OF NA), multivalued, more complex structure
- **molecular_mechanism_evidence**: "types of evidence available to support reported mechanism", according to Data download format txt. complicated structure, is displayed in single-record webpages as a table. Lots of NA. 
  - VERY SPARSE (LOTS OF NA), multivalued, very complex structure with custom terms that aren't defined/explained anywhere yet
- **comments**: treat as free text
- **review** (**new Aug 2025**): indicates when record is under review (according to Data_download_format txt file). Don't know what the possible values are (boolean?) because I've only seen the column be all NA so far.
  - interesting concept idea, but not sure if we want it in Translator or not (also may change in source quickly and we'll be outdated
    
<br>

Can ignore: 
- gene_mim
- gene_symbol
- previous_gene_symbols
- disease_name
- phenotypes: "reported by the publication". Unclear how they fit in gene-disease association or a diff edge (gene-phenotype, phenotype-disease)
- panel: pretty specific, original resource's way of organizing data

In [77]:
## want jsonlines format

import jsonlines

### BioThings-type parser 

Includes `_id` set to g2p_id value

In [ ]:
# ## code chunk for testing parts of inner code

# for row in df.itertuples(index=False):
#     document = {
#         "_id": row.g2p_id,
#         "subject": row.gene_nodenorm_id,
#         "sources": [
#             {
#                 "resource_id": "infores:ebi-gene2phenotype",
#                 "resource_role": "primary_knowledge_source",
#                 "source_record_urls": [row.g2p_record_url]
#             }
#         ],
#         "attributes": [
#             {
#                 "attribute_type_id": "biolink:original_subject",
#                 "value": row.hgnc_id
#             }
#         ]
#     }
#     if pd.notna(row.publications):
#         document["attributes"].append(
#             {
#                 "attribute_type_id": "biolink:publications",
#                 "value": ["PMID:" + i.strip() for i in row.publications.split(";")]
#             }
#         )
#     document
#     break

In [ ]:
# ## put into parser (format): DONE
# ##   don't save in array, yield each document instead

# ## GENERATING DOCS, saving in array
# documents = []

# ## using itertuples because it's faster, preserves datatypes
# for row in df_only_nodenormed.itertuples(index=False):
#     ## simple assignments: no NA or "if"
#     document = {
#         "_id": row.g2p_id,
#         "subject": row.gene_nodenorm_id,
#         "qualifiers": [  ## needs data-modeling/TRAPI validation review
#             {
#                 "qualifier_type_id": "biolink:subject_form_or_variant_qualifier",
#                 "qualifier_value": "genetic_variant_form"
#             }
#         ],
#         "object": row.disease_nodenorm_id,
#         "sources": [
#             {
#                 "resource_id": "infores:ebi-gene2phenotype",
#                 "resource_role": "primary_knowledge_source",
#                 "source_record_urls": [row.g2p_record_url]
#             }
#         ],
#         "attributes": [
#             {
#                 "attribute_type_id": "biolink:knowledge_level",
#                 "value": "knowledge_assertion"
#             },
#             {
#                 "attribute_type_id": "biolink:agent_type",
#                 "value": "manual_agent"
#             },
#             {
#                 "attribute_type_id": "biolink:original_subject",
#                 "original_attribute_name": "hgnc id",  ## original column name
#                 "value": row.hgnc_id
#             },
#             {   ## currently, after NodeNorming, no NAs in OMIM/orphanet column
#                 "attribute_type_id": "biolink:original_object",
#                 "original_attribute_name": "disease mim",  ## original column name
#                 "value": row.disease_mim
#             },
#             {   ## needs data-modeling/TRAPI validation review
#                 ## EBI gene2pheno website calls this "Last Updated"/"Last Updated On"
#                 "attribute_type_id": "biolink:update_date",
#                 "original_attribute_name": "date of last review",  ## original column name
#                 "value": str(row.date_of_last_review)
#             },
#         ]
#     }
    
#     ## more complex assignments ("if", handling NA). When value is NA, list comprehension with split won't work
#     ## predicate
#     if row.confidence == "limited":
#         document["predicate"] = "biolink:related_to"
#     elif row.confidence in ["moderate", "strong", "definitive"]:
#         document["predicate"] = "biolink:causes"
#     else:
#         raise ValueError(f"Unexpected confidence value during predicate mapping: {row.confidence}. Adjust parser.")
#     ## publications
#     if pd.notna(row.publications):
#         document["attributes"].append(
#             {
#                 "attribute_type_id": "biolink:publications",
#                 "value": ["PMID:" + i.strip() for i in row.publications.split(";")]
#             }
#         )
    
#     documents.append(document)

### File: List of TRAPI edges

This code isn't in parser.py

Doesn't have `_id`! Doesn't include original_attribute_name for original_subject, original_object, update_date.

In [78]:
df_only_nodenormed.columns

Index(['g2p_id', 'gene_symbol', 'gene_mim', 'hgnc_id', 'previous_gene_symbols',
       'disease_name', 'disease_mim', 'disease_MONDO', 'allelic_requirement',
       'cross_cutting_modifier', 'confidence', 'variant_consequence',
       'variant_types', 'molecular_mechanism', 'molecular_mechanism_support',
       'molecular_mechanism_categorisation', 'molecular_mechanism_evidence',
       'phenotypes', 'publications', 'panel', 'comments',
       'date_of_last_review', 'review', 'g2p_record_url', 'gene_nodenorm_id',
       'gene_nodenorm_label', 'disease_nodenorm_id', 'disease_nodenorm_label',
       'disease_original_id'],
      dtype='object')

In [80]:
## wrapped with file writer, otherwise contents very similar to before
## commented out original_attribute_name

with jsonlines.open('EBIgene2pheno_trapi_edges.jsonl', mode='w', compact=True) as trapi_writer:

    ## using itertuples because it's faster, preserves datatypes
    for row in df_only_nodenormed.itertuples(index=False):
        
        ## simple assignments: no NA or "if"
        document = {
            "subject": row.gene_nodenorm_id,
            "predicate": "biolink:associated_with",
            "qualifiers": [
                {
                    "qualifier_type_id": "biolink:qualified_predicate",
                    "qualifier_value": "biolink:causes"
                },
                {
                    "qualifier_type_id": "biolink:subject_form_or_variant_qualifier",
                    "qualifier_value": molecularmech_mappings[row.molecular_mechanism]
                },
            ],
            "object": row.disease_nodenorm_id,
            "sources": [
                {
                    "resource_id": "infores:ebi-gene2phenotype",
                    "resource_role": "primary_knowledge_source",
                    "source_record_urls": [row.g2p_record_url]
                }
            ],
            "attributes": [
                {
                    "attribute_type_id": "biolink:knowledge_level",
                    "value": "knowledge_assertion"
                },
                {
                    "attribute_type_id": "biolink:agent_type",
                    "value": "manual_agent"
                },
                {
                    "attribute_type_id": "biolink:original_subject",
#                     "original_attribute_name": "hgnc id",  ## original column name
                    "value": row.hgnc_id
                },
                {   ## uses column created during parsing
                    "attribute_type_id": "biolink:original_object",
                    "value": row.disease_original_id
                },
                ## 2025-09-04: removing right now because common ingest pipeline can't handle it properly
#                 {   ## EBI gene2pheno website calls this "Last Updated"/"Last Updated On"
#                     "attribute_type_id": "biolink:update_date",
# #                     "original_attribute_name": "date of last review",  ## original column name
#                     "value": str(row.date_of_last_review)
#                 },
                {
                    "attribute_type_id": "biolink:allelic_requirement",
                    "value": allelicreq_mappings[row.allelic_requirement]
                },
            ]
        }

        ## more complex assignments ("if", handling NA). When value is NA, list comprehension with split won't work
        ## publications
        if pd.notna(row.publications):
            document["attributes"].append(
                {
                    "attribute_type_id": "biolink:publications",
                    "value": ["PMID:" + i.strip() for i in row.publications.split(";")]
                }
            )
            
        ## doing so it doesn't print
        bytes = trapi_writer.write(document)

### File: KGX edges

This code isn't in parser.py

Doesn't include original_attribute_name for original_subject, original_object, update_date

In [81]:

with jsonlines.open('EBIgene2pheno_kgx_edges.jsonl', mode='w', compact=True) as kgx_edges_writer:

    ## using itertuples because it's faster, preserves datatypes
    for row in df_only_nodenormed.itertuples(index=False):
        
        ## simple assignments: no NA or "if"
        document = {
            "subject": row.gene_nodenorm_id,
            "predicate": "biolink:associated_with",
            "qualified_predicate": "biolink:causes",
            "subject_form_or_variant_qualifier": molecularmech_mappings[row.molecular_mechanism],
            "object": row.disease_nodenorm_id,
            "sources": [
                {
                    "resource_id": "infores:ebi-gene2phenotype",
                    "resource_role": "primary_knowledge_source",
                    "source_record_urls": [row.g2p_record_url]
                }
            ],
            "knowledge_level": "knowledge_assertion",
            "agent_type": "manual_agent",
            "original_subject": row.hgnc_id,
            ## uses column created during parsing
            "original_object": row.disease_original_id,
            ## 2025-09-04: removing right now because common ingest pipeline can't handle it properly
#             ## EBI gene2pheno website calls this "Last Updated"/"Last Updated On"
#             "update_date": str(row.date_of_last_review),
            "allelic_requirement": allelicreq_mappings[row.allelic_requirement],
        }

        ## more complex assignments ("if", handling NA). When value is NA, list comprehension with split won't work
        ## publications
        if pd.notna(row.publications):
            document["publications"] = ["PMID:" + i.strip() for i in row.publications.split(";")]

            
        ## doing so it doesn't print
        bytes = kgx_edges_writer.write(document)

### File: KGX nodes

This code isn't in parser.py

Requires id and category. name and other properties (basically node attributes) are optional. 

In [82]:
nodenormed_genes_final = df_only_nodenormed[["gene_nodenorm_id", "gene_nodenorm_label"]].drop_duplicates()
nodenormed_diseases_final = df_only_nodenormed[["disease_nodenorm_id", "disease_nodenorm_label"]].drop_duplicates()

nodenormed_genes_final
nodenormed_diseases_final

,gene_nodenorm_id,gene_nodenorm_label
0,NCBIGene:3166,HMX1
1,NCBIGene:84464,SLX4
2,NCBIGene:383,ARG1
3,NCBIGene:545,ATR
4,NCBIGene:2187,FANCB
...,...,...
2943,NCBIGene:374462,PTPRQ
2944,NCBIGene:5269,SERPINB6
2945,NCBIGene:7007,TECTA
2947,NCBIGene:286262,TPRN


,disease_nodenorm_id,disease_nodenorm_label
0,MONDO:0012802,oculoauricular syndrome
1,MONDO:0013499,Fanconi anemia complementation group P
2,MONDO:0008814,Argininemia
3,MONDO:0008869,Seckel syndrome 1
4,MONDO:0010351,Fanconi anemia complementation group B
...,...,...
2947,MONDO:0013215,autosomal recessive nonsyndromic hearing loss 79
2948,MONDO:0013963,autosomal recessive nonsyndromic hearing loss 93
2949,MONDO:0007395,craniofacial-deafness-hand syndrome
2950,MONDO:0011519,autosomal dominant nonsyndromic hearing loss 23


In [83]:

with jsonlines.open('EBIgene2pheno_kgx_nodes.jsonl', mode='w', compact=True) as kgx_nodes_writer:
    
    ## using itertuples because it's faster, preserves datatypes
    for row in nodenormed_genes_final.itertuples(index=False):
        ## doing so it doesn't print
        bytes = kgx_nodes_writer.write({
            "id": row.gene_nodenorm_id,
            "name": row.gene_nodenorm_label,            
            ## hard-coded because during pre-NodeNorm process, only kept entities with this primary category
            "category": ["biolink:Gene"]
            
        })

    ## using itertuples because it's faster, preserves datatypes
    for row in nodenormed_diseases_final.itertuples(index=False):
        ## doing so it doesn't print
        bytes = kgx_nodes_writer.write({
            "id": row.disease_nodenorm_id,
            "name": row.disease_nodenorm_label,
            ## hard-coded because during pre-NodeNorm process, only kept entities with this primary category
            "category": ["biolink:Disease"]
        })

In [84]:
nodenormed_genes_final.shape[0] + nodenormed_diseases_final.shape[0]

5209

## DEFUNCT: Checking documents

In [ ]:
# len(documents)

In [ ]:
# df_only_nodenormed.info()

In [ ]:
## code chunk for finding rows -> then look up corresponding doc by idx
# df_only_nodenormed[df_only_nodenormed["disease_mim"].str.contains("orphanet", na=False)]
# df_only_nodenormed[df_only_nodenormed["confidence"] == "limited"]
# df_only_nodenormed[df_only_nodenormed["publications"].isna()]
# df_only_nodenormed[~df_only_nodenormed["publications"].str.contains(";", na=True)]



# df_only_nodenormed[df_only_nodenormed["previous_gene_symbols"].isna()]
# df_only_nodenormed[df_only_nodenormed["disease_MONDO"].notna()]

In [ ]:
# pprint(documents[34])

# documents[416]

## BioThings Parser notes

Fine to use raise/assert in parser (raise is technically better programming behavior: https://realpython.com/python-assert-statement/#understanding-common-pitfalls-of-assert)


My notes on parser:
* adding prefixes to gene/disease IDs is good for pre-NodeNorming steps
* keeping diff gene/disease ID namespaces as separate fields right now is good for current BTE/x-bte-annotation system
  * Also, original subject will always be HGNC, original object will always be disease OMIM with current code


My notes on syntax:
* use `yield` when you want to "return" within a "for loop" (return only happen once, then exit for-loop/function execution)
  * that's what it's used in main execution, when you're iterating over csv rows to generate documents
* use `yield from {function}` to get the data from a generator (created by `yield` being used the function)